In [1]:
import time
from selenium import webdriver
from selenium.common.exceptions import TimeoutException
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.support.ui import WebDriverWait # available since 2.4.0
from selenium.webdriver.support import expected_conditions as EC # available since 2.26.0
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from pywebcopy import save_website
from pywebcopy import WebPage
import pywebcopy
from bs4 import BeautifulSoup
import pandas as pd
pywebcopy.config['bypass_robots'] = True
from datetime import datetime

In [2]:
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager

driver = webdriver.Chrome(ChromeDriverManager().install())

[WDM] - Current google-chrome version is 86.0.4240
WDM        - INFO     - Current google-chrome version is 86.0.4240
[WDM] - Get LATEST driver version for 86.0.4240
WDM        - INFO     - Get LATEST driver version for 86.0.4240
[WDM] - Get LATEST driver version for 86.0.4240
WDM        - INFO     - Get LATEST driver version for 86.0.4240
[WDM] - Trying to download new driver from http://chromedriver.storage.googleapis.com/86.0.4240.22/chromedriver_win32.zip
WDM        - INFO     - Trying to download new driver from http://chromedriver.storage.googleapis.com/86.0.4240.22/chromedriver_win32.zip


[WDM] - Driver has been saved in cache [C:\Users\mpden\.wdm\drivers\chromedriver\win32\86.0.4240.22]
WDM        - INFO     - Driver has been saved in cache [C:\Users\mpden\.wdm\drivers\chromedriver\win32\86.0.4240.22]


In [4]:
driver = webdriver.Chrome()

driver.get('https://www.espn.com/');
time.sleep(8)

##Open Initial Log In Location
search_box = driver.find_element_by_id('global-user-trigger')
search_box.click()
time.sleep(2)
print('Open Log In Tab')

##Click on the Log In location
nextbox = driver.find_element_by_xpath("//a[@data-affiliatename='espn']")
nextbox.click()
print('Click Login')

##Switch to iFrame to enter log in credentials
time.sleep(2)
driver.switch_to.frame("disneyid-iframe")
username = driver.find_element_by_xpath("//input[@placeholder='Username or Email Address']")
print('Switching to iFrame')

##Submit Username and Password
time.sleep(2)
username.send_keys('19dendi')
password = driver.find_element_by_xpath("//input[@placeholder='Password (case sensitive)']")
password.send_keys('espnpassword1234')
time.sleep(2)
print('Logging In')

##Submit credentials
button = driver.find_element_by_xpath("//button[@class='btn btn-primary btn-submit ng-isolate-scope']")
button.click()
driver.page_source

##Open Link Page
time.sleep(8)
search_box = driver.find_element_by_id('global-user-trigger')
search_box.click()
print('Going to Fantasy Link')

##Selecting Fantasy League
time.sleep(2)
leaguego = driver.find_element_by_partial_link_text('Eggbake')
leaguego.click()
print('Entering League')

site = driver.page_source


Open Log In Tab
Click Login
Switching to iFrame
Logging In
Going to Fantasy Link
Entering League


In [5]:
# time.sleep(2)
# leaguego = driver.find_element_by_xpath("//span[@class='jsx-4290203343 week week-range']")
# leaguego.click()
print('Week 9')
#<span class="jsx-4290203343 week week-range">Week 1</span>

Week 9


In [6]:
print('Collecting Eggbake')
time.sleep(8)
page = driver.page_source
soup = BeautifulSoup(page, 'html.parser')
tables = soup.find_all("table")

In [7]:
time.sleep(8)
table = tables[0]
tab_data = [[cell.text for cell in row.find_all(["th","td"])]
                        for row in table.find_all("tr")]
df = pd.DataFrame(tab_data)
gdf = df.drop([0,11,19])

gdf.columns = gdf.iloc[0]
gdf = gdf[1:]
#gdf.drop(columns=['action'])

gdf['Team'] = 'Eggbake'
gdf['Week'] = 9
gdf['Time'] = datetime.now()

In [8]:
nflteams = pd.read_csv('C:\\Users\\mpden\\Documents\\PythonScripts\\FFL\\Data\\static_tables\\teamacc.csv')

In [9]:
time.sleep(8)
nflteams = pd.read_csv('C:\\Users\\mpden\\Documents\\PythonScripts\\FFL\\Data\\static_tables\\teamacc.csv')
eggbakedf = pd.DataFrame(columns = ['Slot', 'Name', 'NFL_Team', 'Pos','Injury_Status', 'Opponent', 'Kickoff', 'Proj', 'Score', 'OPRK', '%Start', "%Owned", "OwnerChange", "PRK", "EstPoints", "Avg", "Last", "FFTeam", "Week", "TimeUpdated"])

for row,col in gdf.iterrows():
    slot = col[0]
    player = col[1]
    opp = col[3]
    kickoff = col[4]
    proj = col[5]
    score = col[6]
    oprk = col[7]
    percstart = col[8]
    percrost = col[9]
    ownchange = col[10]
    prk = col[11]
    projallseason = col[12]
    avg = col[13]
    last = col[14]
    ffteam = col[15]
    week = col[16]
    timeupdated = col[17]
    for team, ac in nflteams.iterrows():
        if len(ac[1])>2 and ("D/ST" not in player) and player[-1] != 'K' and ac[1] in player[-8:]:
            #Replace Team Acronym from Name String
            playpos = player.replace(ac[1], "")
            #Get to Injury status if aparent 
            play = playpos[-3]
            pos = playpos[-2:]
            if play[-1] =='Q':
                #print(f'{playpos[:-3]} and {ac[1]} and Questionable')
                play = playpos[:-3]
                inj = "Q"
                team = ac[1]
                eggbakedf = eggbakedf.append({'Slot': slot, 'Name': play, 'NFL_Team': team, 'Pos': pos,'Injury_Status': inj, 'Opponent':opp, "Kickoff": kickoff, 'Proj': proj, 'Score': score, 'OPRK':oprk, '%Start': percstart, "%Owned": percrost, "OwnerChange":ownchange, "PRK": prk, "EstPoints":projallseason , "Avg": avg, "Last": last, "FFTeam": ffteam, "Week": week, "TimeUpdated": timeupdated},ignore_index=True)
            else:
                #print(f'{playpos[:-2]} and {ac[1]}')
                play = playpos[:-2]
                inj = ""
                team = ac[1]
                eggbakedf = eggbakedf.append({'Slot': slot, 'Name': play, 'NFL_Team': team, 'Pos': pos,'Injury_Status': inj, 'Opponent':opp, "Kickoff": kickoff, 'Proj': proj, 'Score': score, 'OPRK':oprk, '%Start': percstart, "%Owned": percrost, "OwnerChange":ownchange, "PRK": prk, "EstPoints":projallseason , "Avg": avg, "Last": last, "FFTeam": ffteam, "Week": week, "TimeUpdated": timeupdated},ignore_index=True)
        elif len(ac[1])<3 and ("D/ST" not in player) and player[-1] != 'K' and ac[1] in player[-8:]:
            #Replace Team Acronym from Name String
            playpos = player.replace(ac[1], "")
            #Get to Injury status if aparent 
            play = playpos[:-3]
            pos = playpos[-2:]
            if play[-1] =='Q':
                #print(f'{playpos[:-3]} and {ac[1]} and Questionable')
                play = playpos[:-3]
                inj = "Q"
                team = ac[1]
                eggbakedf = eggbakedf.append({'Slot': slot, 'Name': play, 'NFL_Team': team, 'Pos': pos,'Injury_Status': inj, 'Opponent':opp, "Kickoff": kickoff, 'Proj': proj, 'Score': score, 'OPRK':oprk, '%Start': percstart, "%Owned": percrost, "OwnerChange":ownchange, "PRK": prk, "EstPoints":projallseason , "Avg": avg, "Last": last, "FFTeam": ffteam, "Week": week, "TimeUpdated": timeupdated},ignore_index=True)
            else:
                #print(f'{playpos[:-2]} and {ac[1]}')
                play = playpos[:-2]
                inj = ""
                team = ac[1]
                eggbakedf = eggbakedf.append({'Slot': slot, 'Name': play, 'NFL_Team': team, 'Pos': pos,'Injury_Status': inj, 'Opponent':opp, "Kickoff": kickoff, 'Proj': proj, 'Score': score, 'OPRK':oprk, '%Start': percstart, "%Owned": percrost, "OwnerChange":ownchange, "PRK": prk, "EstPoints":projallseason , "Avg": avg, "Last": last, "FFTeam": ffteam, "Week": week, "TimeUpdated": timeupdated},ignore_index=True)
        elif ac[1] in player[-8:] and player[-1] =="K":
            #Replace Team Acronym from Name String
            playpos = player.replace(ac[1], "")
            #Get to Injury status if aparent 
            pos = playpos[-2]
            pos = "K"
            if pos == "Q":
                #print("Questionable")
                inj = "Q"
                team = ac[1]
                eggbakedf = eggbakedf.append({'Slot': slot, 'Name': play, 'NFL_Team': team, 'Pos': pos,'Injury_Status': inj, 'Opponent':opp, "Kickoff": kickoff, 'Proj': proj, 'Score': score, 'OPRK':oprk, '%Start': percstart, "%Owned": percrost, "OwnerChange":ownchange, "PRK": prk, "EstPoints":projallseason , "Avg": avg, "Last": last, "FFTeam": ffteam, "Week": week, "TimeUpdated": timeupdated},ignore_index=True)
            else:
                #print(f'{playpos[:-1]} and {ac[1]}')
                play = playpos[:-1]
                inj = ""
                team = ac[1]
                eggbakedf = eggbakedf.append({'Slot': slot, 'Name': play, 'NFL_Team': team, 'Pos': pos,'Injury_Status': inj, 'Opponent':opp, "Kickoff": kickoff, 'Proj': proj, 'Score': score, 'OPRK':oprk, '%Start': percstart, "%Owned": percrost, "OwnerChange":ownchange, "PRK": prk, "EstPoints":projallseason , "Avg": avg, "Last": last, "FFTeam": ffteam, "Week": week, "TimeUpdated": timeupdated},ignore_index=True)
        elif ac[1] in player[-8:] and "D/ST" in player:
            #Replace Team Acronym from Name String
            playpos = player.replace(ac[1], "")
            play = playpos.replace(" D/ST", " ")
            pos = "D/ST"
            #print(play)
            team = ac[1]
            eggbakedf = eggbakedf.append({'Slot': slot, 'Name': play, 'NFL_Team': team, 'Pos': pos,'Injury_Status': inj, 'Opponent':opp, "Kickoff": kickoff, 'Proj': proj, 'Score': score, 'OPRK':oprk, '%Start': percstart, "%Owned": percrost, "OwnerChange":ownchange, "PRK": prk, "EstPoints":projallseason , "Avg": avg, "Last": last, "FFTeam": ffteam, "Week": week, "TimeUpdated": timeupdated},ignore_index=True)
        elif player == "Empty":
            pass
            play = "Empty"
            ac[1] = ""
            pos = None
    #dasdfdf = dasdfdf.append({'Slot': slot, 'Name': play, 'NFL_Team': team, 'Pos': pos,'Injury_Status': inj, 'Opponent':opp, "Kickoff": kickoff, 'Proj': proj, 'Score': score, 'OPRK':oprk, '%Start': percstart, "%Owned": percrost, "OwnerChange":ownchange, "PRK": prk, "EstPoints":projallseason , "Avg": avg, "Last": last, "FFTeam": ffteam, "Week": week, "TimeUpdated": timeupdated},ignore_index=True)
    #print(f'{slot}, {play}, {ac[1]}')

print('Eggbake Collected')

Eggbake Collected


In [10]:
eggbakedf

,Slot,Name,NFL_Team,Pos,Injury_Status,Opponent,Kickoff,Proj,Score,OPRK,%Start,%Owned,OwnerChange,PRK,EstPoints,Avg,Last,FFTeam,Week,TimeUpdated
0,QB,Tom Brady,TB,QB,,NO,L 3-38,21.9,2.36,18th,73.4,98.3,+2.8,8,172.52,19.2,2.36,Eggbake,9,2020-11-10 09:55:46.426895
1,RB,Aaron Jones,GB,RB,,@SF,W 34-17,11.7,7.9,3rd,81.2,99.8,+0.2,9,104.9,17.5,7.9,Eggbake,9,2020-11-10 09:55:46.426895
2,RB,Latavius Murray,NO,RB,,@TB,W 38-3,6.2,4.2,9th,11.7,61.5,+8.5,38,53.5,6.7,4.2,Eggbake,9,2020-11-10 09:55:46.426895
3,WR,Marquise Brown,Bal,WR,,@Ind,W 24-10,8.1,3.8,11th,40.4,92.5,+3.7,48,53.8,6.7,3.8,Eggbake,9,2020-11-10 09:55:46.426895
4,WR,Mecole Hardman,KC,WR,,Car,W 33-31,7.3,4.8,6th,15.8,50.8,+23.5,37,61.1,6.8,4.8,Eggbake,9,2020-11-10 09:55:46.426895
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
74,IR,Jamaal WilliamsGB,,RB,,@SF,W 34-17,0.0,--,3rd,7.3,69.5,-15.3,30,57.7,8.2,--,Eggbake,9,2020-11-10 09:55:46.426895
75,IR,Jamaal WilliamsGB,,RB,,@SF,W 34-17,0.0,--,3rd,7.3,69.5,-15.3,30,57.7,8.2,--,Eggbake,9,2020-11-10 09:55:46.426895
76,IR,Jamaal WilliamsGB,,RB,,@SF,W 34-17,0.0,--,3rd,7.3,69.5,-15.3,30,57.7,8.2,--,Eggbake,9,2020-11-10 09:55:46.426895
77,IR,Jamaal WilliamsGB,,RB,,@SF,W 34-17,0.0,--,3rd,7.3,69.5,-15.3,30,57.7,8.2,--,Eggbake,9,2020-11-10 09:55:46.426895


In [11]:
path = "C:\\Users\\mpden\\Documents\\PythonScripts\\FFL\\Data\\most_recent\\"

In [12]:
eggbakedf.to_csv("C:\\Users\\mpden\\Documents\\PythonScripts\\FFL\\Data\\most_recent\\eggbake.csv")

# Get Lueders

In [13]:
print('Collecting Lueders')
time.sleep(8)
search_box = driver.find_element_by_xpath("//li[@class = 'teams NavSecondary__Item']")
search_box.click()


##Selecting Fantasy League
time.sleep(2)
leaguego = driver.find_element_by_partial_link_text('Luederberg')
leaguego.click()

# time.sleep(2)
# leaguego = driver.find_element_by_xpath("//span[@class='jsx-4290203343 week week-range']")
# leaguego.click()
print('Week 9')

Week 9


In [14]:
time.sleep(8)
page = driver.page_source
soup = BeautifulSoup(page, 'html.parser')
tables = soup.find_all("table")

table = tables[0]
tab_data = [[cell.text for cell in row.find_all(["th","td"])]
                        for row in table.find_all("tr")]
df = pd.DataFrame(tab_data)
gdf = df.drop([0,11,19])

gdf.columns = gdf.iloc[0]
gdf = gdf[1:]
#gdf.drop(columns=['action'])

gdf['Team'] = 'Das Luederberg'
gdf['Week'] = 9
gdf['Time'] = datetime.now()
gdf
time.sleep(8)

In [15]:
nflteams = pd.read_csv('C:\\Users\\mpden\\Documents\\PythonScripts\\FFL\\Data\\static_tables\\teamacc.csv')
dasdf = pd.DataFrame(columns = ['Slot', 'Name', 'NFL_Team', 'Pos','Injury_Status', 'Opponent', 'Kickoff', 'Proj', 'Score', 'OPRK', '%Start', "%Owned", "OwnerChange", "PRK", "EstPoints", "Avg", "Last", "FFTeam", "Week", "TimeUpdated"])

for row,col in gdf.iterrows():
    slot = col[0]
    player = col[1]
    opp = col[2]
    kickoff = col[3]
    proj = col[4]
    score = col[5]
    oprk = col[6]
    percstart = col[7]
    percrost = col[8]
    ownchange = col[9]
    prk = col[10]
    projallseason = col[11]
    avg = col[12]
    last = col[13]
    ffteam = col[14]
    week = col[15]
    timeupdated = col[16]
    for team, ac in nflteams.iterrows():
        if len(ac[1])>2 and ("D/ST" not in player) and player[-1] != 'K' and ac[1] in player[-8:]:
            #Replace Team Acronym from Name String
            playpos = player.replace(ac[1], "")
            #Get to Injury status if aparent 
            play = playpos[-3]
            pos = playpos[-2:]
            if play[-1] =='Q':
                #print(f'{playpos[:-3]} and {ac[1]} and Questionable')
                play = playpos[:-3]
                inj = "Q"
                team = ac[1]
                dasdf = dasdf.append({'Slot': slot, 'Name': play, 'NFL_Team': team, 'Pos': pos,'Injury_Status': inj, 'Opponent':opp, "Kickoff": kickoff, 'Proj': proj, 'Score': score, 'OPRK':oprk, '%Start': percstart, "%Owned": percrost, "OwnerChange":ownchange, "PRK": prk, "EstPoints":projallseason , "Avg": avg, "Last": last, "FFTeam": ffteam, "Week": week, "TimeUpdated": timeupdated},ignore_index=True)
            else:
                #print(f'{playpos[:-2]} and {ac[1]}')
                play = playpos[:-2]
                inj = ""
                team = ac[1]
                dasdf = dasdf.append({'Slot': slot, 'Name': play, 'NFL_Team': team, 'Pos': pos,'Injury_Status': inj, 'Opponent':opp, "Kickoff": kickoff, 'Proj': proj, 'Score': score, 'OPRK':oprk, '%Start': percstart, "%Owned": percrost, "OwnerChange":ownchange, "PRK": prk, "EstPoints":projallseason , "Avg": avg, "Last": last, "FFTeam": ffteam, "Week": week, "TimeUpdated": timeupdated},ignore_index=True)
        elif len(ac[1])<3 and ("D/ST" not in player) and player[-1] != 'K' and ac[1] in player[-8:]:
            #Replace Team Acronym from Name String
            playpos = player.replace(ac[1], "")
            #Get to Injury status if aparent 
            play = playpos[:-3]
            pos = playpos[-2:]
            if play[-1] =='Q':
                #print(f'{playpos[:-3]} and {ac[1]} and Questionable')
                play = playpos[:-3]
                inj = "Q"
                team = ac[1]
                dasdf = dasdf.append({'Slot': slot, 'Name': play, 'NFL_Team': team, 'Pos': pos,'Injury_Status': inj, 'Opponent':opp, "Kickoff": kickoff, 'Proj': proj, 'Score': score, 'OPRK':oprk, '%Start': percstart, "%Owned": percrost, "OwnerChange":ownchange, "PRK": prk, "EstPoints":projallseason , "Avg": avg, "Last": last, "FFTeam": ffteam, "Week": week, "TimeUpdated": timeupdated},ignore_index=True)
            else:
                #print(f'{playpos[:-2]} and {ac[1]}')
                play = playpos[:-2]
                inj = ""
                team = ac[1]
                dasdf = dasdf.append({'Slot': slot, 'Name': play, 'NFL_Team': team, 'Pos': pos,'Injury_Status': inj, 'Opponent':opp, "Kickoff": kickoff, 'Proj': proj, 'Score': score, 'OPRK':oprk, '%Start': percstart, "%Owned": percrost, "OwnerChange":ownchange, "PRK": prk, "EstPoints":projallseason , "Avg": avg, "Last": last, "FFTeam": ffteam, "Week": week, "TimeUpdated": timeupdated},ignore_index=True)
        elif ac[1] in player[-8:] and player[-1] =="K":
            #Replace Team Acronym from Name String
            playpos = player.replace(ac[1], "")
            #Get to Injury status if aparent 
            pos = playpos[-2]
            pos = "K"
            if pos == "Q":
                #print("Questionable")
                inj = "Q"
                team = ac[1]
                dasdf = dasdf.append({'Slot': slot, 'Name': play, 'NFL_Team': team, 'Pos': pos,'Injury_Status': inj, 'Opponent':opp, "Kickoff": kickoff, 'Proj': proj, 'Score': score, 'OPRK':oprk, '%Start': percstart, "%Owned": percrost, "OwnerChange":ownchange, "PRK": prk, "EstPoints":projallseason , "Avg": avg, "Last": last, "FFTeam": ffteam, "Week": week, "TimeUpdated": timeupdated},ignore_index=True)
            else:
                #print(f'{playpos[:-1]} and {ac[1]}')
                play = playpos[:-1]
                inj = ""
                team = ac[1]
                dasdf = dasdf.append({'Slot': slot, 'Name': play, 'NFL_Team': team, 'Pos': pos,'Injury_Status': inj, 'Opponent':opp, "Kickoff": kickoff, 'Proj': proj, 'Score': score, 'OPRK':oprk, '%Start': percstart, "%Owned": percrost, "OwnerChange":ownchange, "PRK": prk, "EstPoints":projallseason , "Avg": avg, "Last": last, "FFTeam": ffteam, "Week": week, "TimeUpdated": timeupdated},ignore_index=True)
        elif ac[1] in player[-8:] and "D/ST" in player:
            #Replace Team Acronym from Name String
            playpos = player.replace(ac[1], "")
            play = playpos.replace(" D/ST", " ")
            pos = "D/ST"
            #print(play)
            team = ac[1]
            dasdf = dasdf.append({'Slot': slot, 'Name': play, 'NFL_Team': team, 'Pos': pos,'Injury_Status': inj, 'Opponent':opp, "Kickoff": kickoff, 'Proj': proj, 'Score': score, 'OPRK':oprk, '%Start': percstart, "%Owned": percrost, "OwnerChange":ownchange, "PRK": prk, "EstPoints":projallseason , "Avg": avg, "Last": last, "FFTeam": ffteam, "Week": week, "TimeUpdated": timeupdated},ignore_index=True)
        elif player == "Empty":
            pass
            play = "Empty"
            ac[1] = ""
            pos = None
    #dasdfdf = dasdfdf.append({'Slot': slot, 'Name': play, 'NFL_Team': team, 'Pos': pos,'Injury_Status': inj, 'Opponent':opp, "Kickoff": kickoff, 'Proj': proj, 'Score': score, 'OPRK':oprk, '%Start': percstart, "%Owned": percrost, "OwnerChange":ownchange, "PRK": prk, "EstPoints":projallseason , "Avg": avg, "Last": last, "FFTeam": ffteam, "Week": week, "TimeUpdated": timeupdated},ignore_index=True)
    #print(f'{slot}, {play}, {ac[1]}')
dasdf

,Slot,Name,NFL_Team,Pos,Injury_Status,Opponent,Kickoff,Proj,Score,OPRK,%Start,%Owned,OwnerChange,PRK,EstPoints,Avg,Last,FFTeam,Week,TimeUpdated
0,QB,Lamar Jackson,Bal,QB,,@Ind,W 24-10,19.2,18.6,2nd,71.2,99.7,-0.1,11,159.42,19.9,18.6,Das Luederberg,9,2020-11-10 09:57:07.378849
1,RB,Derrick Henry,Ten,RB,,Chi,W 24-17,17.3,6.8,11th,99.1,100.0,0,3,140.4,17.6,6.8,Das Luederberg,9,2020-11-10 09:57:07.378849
2,RB,Damien HarrisQ,NE,RB,,@NYJ,W 30-27,10.5,7.1,26th,31.2,73.5,+42.2,53,43.6,8.7,7.1,Das Luederberg,9,2020-11-10 09:57:07.378849
3,WR,A.J. Brown,Ten,WR,Q,Chi,W 24-17,9.0,16.1,3rd,85.7,99.4,+2.5,19,81.7,13.6,16.1,Das Luederberg,9,2020-11-10 09:57:07.378849
4,WR,DK Metcalf,Sea,WR,,@Buf,L 34-44,11.3,16.8,8th,97.2,99.9,+0.6,2,124.8,15.6,16.8,Das Luederberg,9,2020-11-10 09:57:07.378849
5,TE,Darren Waller,LV,TE,,@LAC,W 31-26,9.9,8.2,28th,98.0,99.8,+0.5,4,61.4,7.7,8.2,Das Luederberg,9,2020-11-10 09:57:07.378849
6,FLEX,Gus Edwards,Bal,RB,,@Ind,W 24-10,7.0,7.4,4th,6.3,18.8,+14,42,49.9,6.2,7.4,Das Luederberg,9,2020-11-10 09:57:07.378849
7,D/ST,Texans D/ST,Hou,D/ST,,@Jax,W 27-25,6.9,1.0,23rd,43.5,50.5,+48.4,30,4.0,0.5,1.0,Das Luederberg,9,2020-11-10 09:57:07.378849
8,K,Wil Lutz,NO,K,,@TB,W 38-3,8.8,8.0,11th,88.7,94.6,+11.3,5,81.0,10.1,8.0,Das Luederberg,9,2020-11-10 09:57:07.378849
9,Bench,CeeDee Lamb,Dal,WR,,Pit,L 19-24,6.2,11.1,25th,14.0,82.0,-0.8,23,78.8,8.8,11.1,Das Luederberg,9,2020-11-10 09:57:07.378849


In [16]:
dasdf.to_csv("C:\\Users\\mpden\\Documents\\PythonScripts\\FFL\\Data\\most_recent\\das.csv")
print('Lueders Collected')

Lueders Collected


# Get Cameron

In [17]:
print('Collecting Cameron')
time.sleep(8)
search_box = driver.find_element_by_xpath("//li[@class = 'teams NavSecondary__Item']")
search_box.click()


##Selecting Fantasy League
time.sleep(2)
leaguego = driver.find_element_by_partial_link_text('Bandits')
leaguego.click()

# time.sleep(2)
# leaguego = driver.find_element_by_xpath("//span[@class='jsx-4290203343 week week-range']")
# leaguego.click()
print('Week 9')

#time.sleep(3)

Week 9


In [18]:
time.sleep(8)
page = driver.page_source
soup = BeautifulSoup(page, 'html.parser')
tables = soup.find_all("table")

table = tables[0]
tab_data = [[cell.text for cell in row.find_all(["th","td"])]
                        for row in table.find_all("tr")]
df = pd.DataFrame(tab_data)
gdf = df.drop([0,11,19])

gdf.columns = gdf.iloc[0]
gdf = gdf[1:]
#gdf.drop(columns=['action'])

gdf['Team'] = 'Banana Bandits'
gdf['Week'] = 9
gdf['Time'] = datetime.now()
gdf
time.sleep(8)

In [19]:
nflteams = pd.read_csv('C:\\Users\\mpden\\Documents\\PythonScripts\\FFL\\Data\\static_tables\\teamacc.csv')
bbdf = pd.DataFrame(columns = ['Slot', 'Name', 'NFL_Team', 'Pos','Injury_Status', 'Opponent', 'Kickoff', 'Proj', 'Score', 'OPRK', '%Start', "%Owned", "OwnerChange", "PRK", "EstPoints", "Avg", "Last", "FFTeam", "Week", "TimeUpdated"])

for row,col in gdf.iterrows():
    slot = col[0]
    player = col[1]
    opp = col[2]
    kickoff = col[3]
    proj = col[4]
    score = col[5]
    oprk = col[6]
    percstart = col[7]
    percrost = col[8]
    ownchange = col[9]
    prk = col[10]
    projallseason = col[11]
    avg = col[12]
    last = col[13]
    ffteam = col[14]
    week = col[15]
    timeupdated = col[16]
    for team, ac in nflteams.iterrows():
        if len(ac[1])>2 and ("D/ST" not in player) and player[-1] != 'K' and ac[1] in player[-8:]:
            #Replace Team Acronym from Name String
            playpos = player.replace(ac[1], "")
            #Get to Injury status if aparent 
            play = playpos[-3]
            pos = playpos[-2:]
            if play[-1] =='Q':
                #print(f'{playpos[:-3]} and {ac[1]} and Questionable')
                play = playpos[:-3]
                inj = "Q"
                team = ac[1]
                bbdf = bbdf.append({'Slot': slot, 'Name': play, 'NFL_Team': team, 'Pos': pos,'Injury_Status': inj, 'Opponent':opp, "Kickoff": kickoff, 'Proj': proj, 'Score': score, 'OPRK':oprk, '%Start': percstart, "%Owned": percrost, "OwnerChange":ownchange, "PRK": prk, "EstPoints":projallseason , "Avg": avg, "Last": last, "FFTeam": ffteam, "Week": week, "TimeUpdated": timeupdated},ignore_index=True)
            else:
                #print(f'{playpos[:-2]} and {ac[1]}')
                play = playpos[:-2]
                inj = ""
                team = ac[1]
                bbdf = bbdf.append({'Slot': slot, 'Name': play, 'NFL_Team': team, 'Pos': pos,'Injury_Status': inj, 'Opponent':opp, "Kickoff": kickoff, 'Proj': proj, 'Score': score, 'OPRK':oprk, '%Start': percstart, "%Owned": percrost, "OwnerChange":ownchange, "PRK": prk, "EstPoints":projallseason , "Avg": avg, "Last": last, "FFTeam": ffteam, "Week": week, "TimeUpdated": timeupdated},ignore_index=True)
        elif len(ac[1])<3 and ("D/ST" not in player) and player[-1] != 'K' and ac[1] in player[-8:]:
            #Replace Team Acronym from Name String
            playpos = player.replace(ac[1], "")
            #Get to Injury status if aparent 
            play = playpos[:-3]
            pos = playpos[-2:]
            if play[-1] =='Q':
                #print(f'{playpos[:-3]} and {ac[1]} and Questionable')
                play = playpos[:-3]
                inj = "Q"
                team = ac[1]
                bbdf = bbdf.append({'Slot': slot, 'Name': play, 'NFL_Team': team, 'Pos': pos,'Injury_Status': inj, 'Opponent':opp, "Kickoff": kickoff, 'Proj': proj, 'Score': score, 'OPRK':oprk, '%Start': percstart, "%Owned": percrost, "OwnerChange":ownchange, "PRK": prk, "EstPoints":projallseason , "Avg": avg, "Last": last, "FFTeam": ffteam, "Week": week, "TimeUpdated": timeupdated},ignore_index=True)
            else:
                #print(f'{playpos[:-2]} and {ac[1]}')
                play = playpos[:-2]
                inj = ""
                team = ac[1]
                bbdf = bbdf.append({'Slot': slot, 'Name': play, 'NFL_Team': team, 'Pos': pos,'Injury_Status': inj, 'Opponent':opp, "Kickoff": kickoff, 'Proj': proj, 'Score': score, 'OPRK':oprk, '%Start': percstart, "%Owned": percrost, "OwnerChange":ownchange, "PRK": prk, "EstPoints":projallseason , "Avg": avg, "Last": last, "FFTeam": ffteam, "Week": week, "TimeUpdated": timeupdated},ignore_index=True)
        elif ac[1] in player[-8:] and player[-1] =="K":
            #Replace Team Acronym from Name String
            playpos = player.replace(ac[1], "")
            #Get to Injury status if aparent 
            pos = playpos[-2]
            pos = "K"
            if pos == "Q":
                #print("Questionable")
                inj = "Q"
                team = ac[1]
                bbdf = bbdf.append({'Slot': slot, 'Name': play, 'NFL_Team': team, 'Pos': pos,'Injury_Status': inj, 'Opponent':opp, "Kickoff": kickoff, 'Proj': proj, 'Score': score, 'OPRK':oprk, '%Start': percstart, "%Owned": percrost, "OwnerChange":ownchange, "PRK": prk, "EstPoints":projallseason , "Avg": avg, "Last": last, "FFTeam": ffteam, "Week": week, "TimeUpdated": timeupdated},ignore_index=True)
            else:
                #print(f'{playpos[:-1]} and {ac[1]}')
                play = playpos[:-1]
                inj = ""
                team = ac[1]
                bbdf = bbdf.append({'Slot': slot, 'Name': play, 'NFL_Team': team, 'Pos': pos,'Injury_Status': inj, 'Opponent':opp, "Kickoff": kickoff, 'Proj': proj, 'Score': score, 'OPRK':oprk, '%Start': percstart, "%Owned": percrost, "OwnerChange":ownchange, "PRK": prk, "EstPoints":projallseason , "Avg": avg, "Last": last, "FFTeam": ffteam, "Week": week, "TimeUpdated": timeupdated},ignore_index=True)
        elif ac[1] in player[-8:] and "D/ST" in player:
            #Replace Team Acronym from Name String
            playpos = player.replace(ac[1], "")
            play = playpos.replace(" D/ST", " ")
            pos = "D/ST"
            #print(play)
            team = ac[1]
            bbdf = bbdf.append({'Slot': slot, 'Name': play, 'NFL_Team': team, 'Pos': pos,'Injury_Status': inj, 'Opponent':opp, "Kickoff": kickoff, 'Proj': proj, 'Score': score, 'OPRK':oprk, '%Start': percstart, "%Owned": percrost, "OwnerChange":ownchange, "PRK": prk, "EstPoints":projallseason , "Avg": avg, "Last": last, "FFTeam": ffteam, "Week": week, "TimeUpdated": timeupdated},ignore_index=True)
        elif player == "Empty":
            pass
            play = "Empty"
            ac[1] = ""
            pos = None
    #bbdf = bbdf.append({'Slot': slot, 'Name': play, 'NFL_Team': team, 'Pos': pos,'Injury_Status': inj, 'Opponent':opp, "Kickoff": kickoff, 'Proj': proj, 'Score': score, 'OPRK':oprk, '%Start': percstart, "%Owned": percrost, "OwnerChange":ownchange, "PRK": prk, "EstPoints":projallseason , "Avg": avg, "Last": last, "FFTeam": ffteam, "Week": week, "TimeUpdated": timeupdated},ignore_index=True)
    #print(f'{slot}, {play}, {ac[1]}')
    
bbdf

,Slot,Name,NFL_Team,Pos,Injury_Status,Opponent,Kickoff,Proj,Score,OPRK,%Start,%Owned,OwnerChange,PRK,EstPoints,Avg,Last,FFTeam,Week,TimeUpdated
0,QB,Russell Wilson,Sea,QB,,@Buf,L 34-44,25.6,22.1,19th,96.4,100.0,+0.2,3,228.14,28.5,22.1,Banana Bandits,9,2020-11-10 09:58:12.591830
1,RB,David Johnson,Hou,RB,Q,@Jax,W 27-25,14.0,1.6,28th,81.0,99.0,+3.8,20,78.9,9.9,1.6,Banana Bandits,9,2020-11-10 09:58:12.591830
2,RB,Todd Gurley II,Atl,RB,,Den,W 34-27,13.5,11.5,8th,61.6,99.7,+0.7,5,120.3,13.4,11.5,Banana Bandits,9,2020-11-10 09:58:12.591830
3,WR,Mike Evans,TB,WR,,NO,L 3-38,8.5,6.4,23rd,76.8,99.1,+0.4,15,85.7,9.5,6.4,Banana Bandits,9,2020-11-10 09:58:12.591830
4,WR,Keenan Allen,LAC,WR,,LV,L 26-31,12.1,16.3,18th,95.0,99.8,+1.4,13,87.1,10.9,16.3,Banana Bandits,9,2020-11-10 09:58:12.591830
5,TE,Jonnu Smith,Ten,TE,,Chi,W 24-17,5.5,9.2,29th,47.6,86.4,-2,3,66.4,8.3,9.2,Banana Bandits,9,2020-11-10 09:58:12.591830
6,FLEX,Christian McCaffrey,Car,RB,Q,@KC,L 31-33,14.5,27.1,19th,90.0,100.0,+0.1,24,73.4,24.5,27.1,Banana Bandits,9,2020-11-10 09:58:12.591830
7,D/ST,Patriots D/ST,NE,D/ST,Q,@NYJ,W 30-27,6.3,3.0,32nd,79.0,92.7,+20.1,13,47.0,5.9,3.0,Banana Bandits,9,2020-11-10 09:58:12.591830
8,K,Mason Crosby,GB,K,,@SF,W 34-17,6.9,12.0,25th,31.3,40.8,-6.6,15,69.0,8.6,12.0,Banana Bandits,9,2020-11-10 09:58:12.591830
9,Bench,DJ Moore,Car,WR,,@KC,L 31-33,8.3,1.8,4th,68.8,98.2,+1.2,17,82.0,9.1,1.8,Banana Bandits,9,2020-11-10 09:58:12.591830


In [20]:
bbdf.to_csv("C:\\Users\\mpden\\Documents\\PythonScripts\\FFL\\Data\\most_recent\\bb.csv")
print('Cameron Collected')

Cameron Collected


# Get Greg

In [21]:
print('Collecting Greg')
time.sleep(8)
search_box = driver.find_element_by_xpath("//li[@class = 'teams NavSecondary__Item']")
search_box.click()


##Selecting Fantasy League
time.sleep(2)
leaguego = driver.find_element_by_partial_link_text('Gregor')
leaguego.click()

# time.sleep(2)
# leaguego = driver.find_element_by_xpath("//span[@class='jsx-4290203343 week week-range']")
# leaguego.click()
print('Week 9')

#time.sleep(3)

Week 9


In [22]:
time.sleep(8)
page = driver.page_source
soup = BeautifulSoup(page, 'html.parser')
tables = soup.find_all("table")

table = tables[0]
tab_data = [[cell.text for cell in row.find_all(["th","td"])]
                        for row in table.find_all("tr")]
df = pd.DataFrame(tab_data)
gdf = df.drop([0,11,19])

gdf.columns = gdf.iloc[0]
gdf = gdf[1:]
#gdf.drop(columns=['action'])

gdf['Team'] = 'Gregoriuos Rex'
gdf['Week'] = 9
gdf['Time'] = datetime.now()
gdf

1,SLOT,Player,opp,STATUS,proj,SCORE,OPRK,%ST,%ROST,+/-,PRK,FPTS,avg,LAST,Team,Week,Time
2,QB,Matt RyanAtlQB,Den,W 34-27,18.1,22.36,25th,43.8,91.4,+5.5,7,172.64,19.2,22.36,Gregoriuos Rex,9,2020-11-10 09:59:10.974709
3,RB,Rex BurkheadNERB,@NYJ,W 30-27,6.4,12.7,26th,9.4,27.3,+3.4,41,62.8,7.9,12.7,Gregoriuos Rex,9,2020-11-10 09:59:10.974709
4,RB,Melvin Gordon IIIDenRB,@Atl,L 27-34,10.6,2.7,6th,72.7,97.7,+3.8,25,72.0,10.3,2.7,Gregoriuos Rex,9,2020-11-10 09:59:10.974709
5,WR,Emmanuel SandersNOWR,@TB,W 38-3,6.9,9.8,9th,22.3,72.1,+19.6,50,52.2,8.7,9.8,Gregoriuos Rex,9,2020-11-10 09:59:10.974709
6,WR,Cole BeasleyBufWR,Sea,W 44-34,6.9,3.9,32nd,33.0,70.5,+14.5,33,65.3,7.3,3.9,Gregoriuos Rex,9,2020-11-10 09:59:10.974709
7,TE,Hayden HurstAtlTE,Den,W 34-27,6.1,6.2,9th,42.0,84.9,+14.2,9,59.1,6.6,6.2,Gregoriuos Rex,9,2020-11-10 09:59:10.974709
8,FLEX,J.D. McKissicWshRB,NYG,L 20-23,6.8,8.2,15th,23.1,47.9,+16,48,39.6,5.0,8.2,Gregoriuos Rex,9,2020-11-10 09:59:10.974709
9,D/ST,Washington D/STWshD/ST,NYG,L 20-23,7.4,4.0,30th,70.0,82.9,+65.4,12,48.0,6.0,4.0,Gregoriuos Rex,9,2020-11-10 09:59:10.974709
10,K,Graham GanoNYGK,@Wsh,W 23-20,7.9,13.0,18th,14.0,19.4,-2,5,81.0,9.0,13.0,Gregoriuos Rex,9,2020-11-10 09:59:10.974709
12,Bench,Cooper KuppLARWR,*BYE*,--,--,--,--,39.8,98.2,+2.4,32,66.0,8.3,--,Gregoriuos Rex,9,2020-11-10 09:59:10.974709


In [23]:
nflteams = pd.read_csv('C:\\Users\\mpden\\Documents\\PythonScripts\\FFL\\Data\\static_tables\\teamacc.csv')
grdf = pd.DataFrame(columns = ['Slot', 'Name', 'NFL_Team', 'Pos','Injury_Status', 'Opponent', 'Kickoff', 'Proj', 'Score', 'OPRK', '%Start', "%Owned", "OwnerChange", "PRK", "EstPoints", "Avg", "Last", "FFTeam", "Week", "TimeUpdated"])

for row,col in gdf.iterrows():
    slot = col[0]
    player = col[1]
    opp = col[2]
    kickoff = col[3]
    proj = col[4]
    score = col[5]
    oprk = col[6]
    percstart = col[7]
    percrost = col[8]
    ownchange = col[9]
    prk = col[10]
    projallseason = col[11]
    avg = col[12]
    last = col[13]
    ffteam = col[14]
    week = col[15]
    timeupdated = col[16]
    for team, ac in nflteams.iterrows():
        if len(ac[1])>2 and ("D/ST" not in player) and player[-1] != 'K' and ac[1] in player[-8:]:
            #Replace Team Acronym from Name String
            playpos = player.replace(ac[1], "")
            #Get to Injury status if aparent 
            play = playpos[-3]
            pos = playpos[-2:]
            if play[-1] =='Q':
                #print(f'{playpos[:-3]} and {ac[1]} and Questionable')
                play = playpos[:-3]
                inj = "Q"
                team = ac[1]
                grdf = grdf.append({'Slot': slot, 'Name': play, 'NFL_Team': team, 'Pos': pos,'Injury_Status': inj, 'Opponent':opp, "Kickoff": kickoff, 'Proj': proj, 'Score': score, 'OPRK':oprk, '%Start': percstart, "%Owned": percrost, "OwnerChange":ownchange, "PRK": prk, "EstPoints":projallseason , "Avg": avg, "Last": last, "FFTeam": ffteam, "Week": week, "TimeUpdated": timeupdated},ignore_index=True)
            else:
                #print(f'{playpos[:-2]} and {ac[1]}')
                play = playpos[:-2]
                inj = ""
                team = ac[1]
                grdf = grdf.append({'Slot': slot, 'Name': play, 'NFL_Team': team, 'Pos': pos,'Injury_Status': inj, 'Opponent':opp, "Kickoff": kickoff, 'Proj': proj, 'Score': score, 'OPRK':oprk, '%Start': percstart, "%Owned": percrost, "OwnerChange":ownchange, "PRK": prk, "EstPoints":projallseason , "Avg": avg, "Last": last, "FFTeam": ffteam, "Week": week, "TimeUpdated": timeupdated},ignore_index=True)
        elif len(ac[1])<3 and ("D/ST" not in player) and player[-1] != 'K' and ac[1] in player[-8:]:
            #Replace Team Acronym from Name String
            playpos = player.replace(ac[1], "")
            #Get to Injury status if aparent 
            play = playpos[:-3]
            pos = playpos[-2:]
            if play[-1] =='Q':
                #print(f'{playpos[:-3]} and {ac[1]} and Questionable')
                play = playpos[:-3]
                inj = "Q"
                team = ac[1]
                grdf = grdf.append({'Slot': slot, 'Name': play, 'NFL_Team': team, 'Pos': pos,'Injury_Status': inj, 'Opponent':opp, "Kickoff": kickoff, 'Proj': proj, 'Score': score, 'OPRK':oprk, '%Start': percstart, "%Owned": percrost, "OwnerChange":ownchange, "PRK": prk, "EstPoints":projallseason , "Avg": avg, "Last": last, "FFTeam": ffteam, "Week": week, "TimeUpdated": timeupdated},ignore_index=True)
            else:
                #print(f'{playpos[:-2]} and {ac[1]}')
                play = playpos[:-2]
                inj = ""
                team = ac[1]
                grdf = grdf.append({'Slot': slot, 'Name': play, 'NFL_Team': team, 'Pos': pos,'Injury_Status': inj, 'Opponent':opp, "Kickoff": kickoff, 'Proj': proj, 'Score': score, 'OPRK':oprk, '%Start': percstart, "%Owned": percrost, "OwnerChange":ownchange, "PRK": prk, "EstPoints":projallseason , "Avg": avg, "Last": last, "FFTeam": ffteam, "Week": week, "TimeUpdated": timeupdated},ignore_index=True)
        elif ac[1] in player[-8:] and player[-1] =="K":
            #Replace Team Acronym from Name String
            playpos = player.replace(ac[1], "")
            #Get to Injury status if aparent 
            pos = playpos[-2]
            pos = "K"
            if pos == "Q":
                #print("Questionable")
                inj = "Q"
                team = ac[1]
                grdf = grdf.append({'Slot': slot, 'Name': play, 'NFL_Team': team, 'Pos': pos,'Injury_Status': inj, 'Opponent':opp, "Kickoff": kickoff, 'Proj': proj, 'Score': score, 'OPRK':oprk, '%Start': percstart, "%Owned": percrost, "OwnerChange":ownchange, "PRK": prk, "EstPoints":projallseason , "Avg": avg, "Last": last, "FFTeam": ffteam, "Week": week, "TimeUpdated": timeupdated},ignore_index=True)
            else:
                #print(f'{playpos[:-1]} and {ac[1]}')
                play = playpos[:-1]
                inj = ""
                team = ac[1]
                grdf = grdf.append({'Slot': slot, 'Name': play, 'NFL_Team': team, 'Pos': pos,'Injury_Status': inj, 'Opponent':opp, "Kickoff": kickoff, 'Proj': proj, 'Score': score, 'OPRK':oprk, '%Start': percstart, "%Owned": percrost, "OwnerChange":ownchange, "PRK": prk, "EstPoints":projallseason , "Avg": avg, "Last": last, "FFTeam": ffteam, "Week": week, "TimeUpdated": timeupdated},ignore_index=True)
        elif ac[1] in player[-8:] and "D/ST" in player:
            #Replace Team Acronym from Name String
            playpos = player.replace(ac[1], "")
            play = playpos.replace(" D/ST", " ")
            pos = "D/ST"
            #print(play)
            team = ac[1]
            grdf = grdf.append({'Slot': slot, 'Name': play, 'NFL_Team': team, 'Pos': pos,'Injury_Status': inj, 'Opponent':opp, "Kickoff": kickoff, 'Proj': proj, 'Score': score, 'OPRK':oprk, '%Start': percstart, "%Owned": percrost, "OwnerChange":ownchange, "PRK": prk, "EstPoints":projallseason , "Avg": avg, "Last": last, "FFTeam": ffteam, "Week": week, "TimeUpdated": timeupdated},ignore_index=True)
        elif player == "Empty":
            pass
            play = "Empty"
            ac[1] = ""
            pos = None
    #bbdf = bbdf.append({'Slot': slot, 'Name': play, 'NFL_Team': team, 'Pos': pos,'Injury_Status': inj, 'Opponent':opp, "Kickoff": kickoff, 'Proj': proj, 'Score': score, 'OPRK':oprk, '%Start': percstart, "%Owned": percrost, "OwnerChange":ownchange, "PRK": prk, "EstPoints":projallseason , "Avg": avg, "Last": last, "FFTeam": ffteam, "Week": week, "TimeUpdated": timeupdated},ignore_index=True)
    #print(f'{slot}, {play}, {ac[1]}')
    
grdf

,Slot,Name,NFL_Team,Pos,Injury_Status,Opponent,Kickoff,Proj,Score,OPRK,%Start,%Owned,OwnerChange,PRK,EstPoints,Avg,Last,FFTeam,Week,TimeUpdated
0,QB,Matt Ryan,Atl,QB,,Den,W 34-27,18.1,22.36,25th,43.8,91.4,+5.5,7,172.64,19.2,22.36,Gregoriuos Rex,9,2020-11-10 09:59:10.974709
1,RB,Rex Burkhead,NE,RB,,@NYJ,W 30-27,6.4,12.7,26th,9.4,27.3,+3.4,41,62.8,7.9,12.7,Gregoriuos Rex,9,2020-11-10 09:59:10.974709
2,RB,Melvin Gordon III,Den,RB,,@Atl,L 27-34,10.6,2.7,6th,72.7,97.7,+3.8,25,72.0,10.3,2.7,Gregoriuos Rex,9,2020-11-10 09:59:10.974709
3,WR,Emmanuel Sanders,NO,WR,,@TB,W 38-3,6.9,9.8,9th,22.3,72.1,+19.6,50,52.2,8.7,9.8,Gregoriuos Rex,9,2020-11-10 09:59:10.974709
4,WR,Cole Beasley,Buf,WR,,Sea,W 44-34,6.9,3.9,32nd,33.0,70.5,+14.5,33,65.3,7.3,3.9,Gregoriuos Rex,9,2020-11-10 09:59:10.974709
5,TE,Hayden Hurst,Atl,TE,,Den,W 34-27,6.1,6.2,9th,42.0,84.9,+14.2,9,59.1,6.6,6.2,Gregoriuos Rex,9,2020-11-10 09:59:10.974709
6,FLEX,J.D. McKissic,Wsh,RB,,NYG,L 20-23,6.8,8.2,15th,23.1,47.9,+16,48,39.6,5.0,8.2,Gregoriuos Rex,9,2020-11-10 09:59:10.974709
7,D/ST,Washington D/ST,Wsh,D/ST,,NYG,L 20-23,7.4,4.0,30th,70.0,82.9,+65.4,12,48.0,6.0,4.0,Gregoriuos Rex,9,2020-11-10 09:59:10.974709
8,K,Graham Gano,NYG,K,,@Wsh,W 23-20,7.9,13.0,18th,14.0,19.4,-2,5,81.0,9.0,13.0,Gregoriuos Rex,9,2020-11-10 09:59:10.974709
9,Bench,Cooper Kupp,LAR,WR,,*BYE*,--,--,--,--,39.8,98.2,+2.4,32,66.0,8.3,--,Gregoriuos Rex,9,2020-11-10 09:59:10.974709


In [24]:
grdf.to_csv("C:\\Users\\mpden\\Documents\\PythonScripts\\FFL\\Data\\most_recent\\gr.csv")
print("Greg Collected")

Greg Collected


# Get Tony

In [25]:
print("Collecting Tony")
time.sleep(8)
search_box = driver.find_element_by_xpath("//li[@class = 'teams NavSecondary__Item']")
search_box.click()


##Selecting Fantasy League
time.sleep(2)
leaguego = driver.find_element_by_partial_link_text('Sunshine')
leaguego.click()

# time.sleep(2)
# leaguego = driver.find_element_by_xpath("//span[@class='jsx-4290203343 week week-range']")
# leaguego.click()
print('Week 9')

# time.sleep(3)

Week 9


In [26]:
time.sleep(8)
page = driver.page_source
soup = BeautifulSoup(page, 'html.parser')
tables = soup.find_all("table")

table = tables[0]
tab_data = [[cell.text for cell in row.find_all(["th","td"])]
                        for row in table.find_all("tr")]
df = pd.DataFrame(tab_data)
gdf = df.drop([0,11,19])

gdf.columns = gdf.iloc[0]
gdf = gdf[1:]
#gdf.drop(columns=['action'])

gdf['Team'] = 'Team Sunshine'
gdf['Week'] = 9
gdf['Time'] = datetime.now()
gdf

1,SLOT,Player,opp,STATUS,proj,SCORE,OPRK,%ST,%ROST,+/-,PRK,FPTS,avg,LAST,Team,Week,Time
2,QB,Justin HerbertLACQB,LV,L 26-31,21.9,23.44,28th,71.1,96.5,+13.8,9,170.44,24.3,23.44,Team Sunshine,9,2020-11-10 10:00:01.708916
3,RB,Ezekiel ElliottDalRB,Pit,L 19-24,10.9,6.9,5th,43.6,99.9,0,6,111.0,12.3,6.9,Team Sunshine,9,2020-11-10 10:00:01.708916
4,RB,JaMycal HastySFRB,GB,L 17-34,12.5,1.3,31st,38.7,80.8,+35.1,71,23.2,3.3,1.3,Team Sunshine,9,2020-11-10 10:00:01.708916
5,WR,Julio JonesAtlWR,Den,W 34-27,12.2,11.4,24th,58.9,99.9,0,18,81.8,11.7,11.4,Team Sunshine,9,2020-11-10 10:00:01.708916
6,WR,DeAndre HopkinsAriWR,Mia,L 31-34,11.1,3.0,20th,96.8,100.0,0,9,89.4,11.2,3.0,Team Sunshine,9,2020-11-10 10:00:01.708916
7,TE,Hunter HenryLACTE,LV,L 26-31,7.2,3.3,12th,69.9,94.5,+3.8,18,41.7,5.2,3.3,Team Sunshine,9,2020-11-10 10:00:01.708916
8,FLEX,Brandin CooksHouWR,@Jax,W 27-25,8.5,14.3,22nd,59.3,92.2,+14.7,28,69.0,8.6,14.3,Team Sunshine,9,2020-11-10 10:00:01.708916
9,D/ST,Ravens D/STBalD/ST,@Ind,W 24-10,6.4,13.0,15th,88.2,98.4,+3.6,2,84.0,10.5,13.0,Team Sunshine,9,2020-11-10 10:00:01.708916
10,K,Greg ZuerleinDalK,Pit,L 19-24,7.1,15.0,6th,36.7,42.6,-10.2,13,74.0,8.2,15.0,Team Sunshine,9,2020-11-10 10:00:01.708916
12,Bench,Devin SingletaryBufRB,Sea,W 44-34,8.0,3.4,13th,43.8,91.3,+3.9,29,63.8,7.1,3.4,Team Sunshine,9,2020-11-10 10:00:01.708916


In [27]:
nflteams = pd.read_csv('C:\\Users\\mpden\\Documents\\PythonScripts\\FFL\\Data\\static_tables\\teamacc.csv')
ssdf = pd.DataFrame(columns = ['Slot', 'Name', 'NFL_Team', 'Pos','Injury_Status', 'Opponent', 'Kickoff', 'Proj', 'Score', 'OPRK', '%Start', "%Owned", "OwnerChange", "PRK", "EstPoints", "Avg", "Last", "FFTeam", "Week", "TimeUpdated"])

for row,col in gdf.iterrows():
    slot = col[0]
    player = col[1]
    opp = col[2]
    kickoff = col[3]
    proj = col[4]
    score = col[5]
    oprk = col[6]
    percstart = col[7]
    percrost = col[8]
    ownchange = col[9]
    prk = col[10]
    projallseason = col[11]
    avg = col[12]
    last = col[13]
    ffteam = col[14]
    week = col[15]
    timeupdated = col[16]
    for team, ac in nflteams.iterrows():
        if len(ac[1])>2 and ("D/ST" not in player) and player[-1] != 'K' and ac[1] in player[-8:]:
            #Replace Team Acronym from Name String
            playpos = player.replace(ac[1], "")
            #Get to Injury status if aparent 
            play = playpos[-3]
            pos = playpos[-2:]
            if play[-1] =='Q':
                #print(f'{playpos[:-3]} and {ac[1]} and Questionable')
                play = playpos[:-3]
                inj = "Q"
                team = ac[1]
                ssdf = ssdf.append({'Slot': slot, 'Name': play, 'NFL_Team': team, 'Pos': pos,'Injury_Status': inj, 'Opponent':opp, "Kickoff": kickoff, 'Proj': proj, 'Score': score, 'OPRK':oprk, '%Start': percstart, "%Owned": percrost, "OwnerChange":ownchange, "PRK": prk, "EstPoints":projallseason , "Avg": avg, "Last": last, "FFTeam": ffteam, "Week": week, "TimeUpdated": timeupdated},ignore_index=True)
            else:
                #print(f'{playpos[:-2]} and {ac[1]}')
                play = playpos[:-2]
                inj = ""
                team = ac[1]
                ssdf = ssdf.append({'Slot': slot, 'Name': play, 'NFL_Team': team, 'Pos': pos,'Injury_Status': inj, 'Opponent':opp, "Kickoff": kickoff, 'Proj': proj, 'Score': score, 'OPRK':oprk, '%Start': percstart, "%Owned": percrost, "OwnerChange":ownchange, "PRK": prk, "EstPoints":projallseason , "Avg": avg, "Last": last, "FFTeam": ffteam, "Week": week, "TimeUpdated": timeupdated},ignore_index=True)
        elif len(ac[1])<3 and ("D/ST" not in player) and player[-1] != 'K' and ac[1] in player[-8:]:
            #Replace Team Acronym from Name String
            playpos = player.replace(ac[1], "")
            #Get to Injury status if aparent 
            play = playpos[:-3]
            pos = playpos[-2:]
            if play[-1] =='Q':
                #print(f'{playpos[:-3]} and {ac[1]} and Questionable')
                play = playpos[:-3]
                inj = "Q"
                team = ac[1]
                ssdf = ssdf.append({'Slot': slot, 'Name': play, 'NFL_Team': team, 'Pos': pos,'Injury_Status': inj, 'Opponent':opp, "Kickoff": kickoff, 'Proj': proj, 'Score': score, 'OPRK':oprk, '%Start': percstart, "%Owned": percrost, "OwnerChange":ownchange, "PRK": prk, "EstPoints":projallseason , "Avg": avg, "Last": last, "FFTeam": ffteam, "Week": week, "TimeUpdated": timeupdated},ignore_index=True)
            else:
                #print(f'{playpos[:-2]} and {ac[1]}')
                play = playpos[:-2]
                inj = ""
                team = ac[1]
                ssdf = ssdf.append({'Slot': slot, 'Name': play, 'NFL_Team': team, 'Pos': pos,'Injury_Status': inj, 'Opponent':opp, "Kickoff": kickoff, 'Proj': proj, 'Score': score, 'OPRK':oprk, '%Start': percstart, "%Owned": percrost, "OwnerChange":ownchange, "PRK": prk, "EstPoints":projallseason , "Avg": avg, "Last": last, "FFTeam": ffteam, "Week": week, "TimeUpdated": timeupdated},ignore_index=True)
        elif ac[1] in player[-8:] and player[-1] =="K":
            #Replace Team Acronym from Name String
            playpos = player.replace(ac[1], "")
            #Get to Injury status if aparent 
            pos = playpos[-2]
            pos = "K"
            if pos == "Q":
                #print("Questionable")
                inj = "Q"
                team = ac[1]
                ssdf = ssdf.append({'Slot': slot, 'Name': play, 'NFL_Team': team, 'Pos': pos,'Injury_Status': inj, 'Opponent':opp, "Kickoff": kickoff, 'Proj': proj, 'Score': score, 'OPRK':oprk, '%Start': percstart, "%Owned": percrost, "OwnerChange":ownchange, "PRK": prk, "EstPoints":projallseason , "Avg": avg, "Last": last, "FFTeam": ffteam, "Week": week, "TimeUpdated": timeupdated},ignore_index=True)
            else:
                #print(f'{playpos[:-1]} and {ac[1]}')
                play = playpos[:-1]
                inj = ""
                team = ac[1]
                ssdf = ssdf.append({'Slot': slot, 'Name': play, 'NFL_Team': team, 'Pos': pos,'Injury_Status': inj, 'Opponent':opp, "Kickoff": kickoff, 'Proj': proj, 'Score': score, 'OPRK':oprk, '%Start': percstart, "%Owned": percrost, "OwnerChange":ownchange, "PRK": prk, "EstPoints":projallseason , "Avg": avg, "Last": last, "FFTeam": ffteam, "Week": week, "TimeUpdated": timeupdated},ignore_index=True)
        elif ac[1] in player[-8:] and "D/ST" in player:
            #Replace Team Acronym from Name String
            playpos = player.replace(ac[1], "")
            play = playpos.replace(" D/ST", " ")
            pos = "D/ST"
            #print(play)
            team = ac[1]
            ssdf = ssdf.append({'Slot': slot, 'Name': play, 'NFL_Team': team, 'Pos': pos,'Injury_Status': inj, 'Opponent':opp, "Kickoff": kickoff, 'Proj': proj, 'Score': score, 'OPRK':oprk, '%Start': percstart, "%Owned": percrost, "OwnerChange":ownchange, "PRK": prk, "EstPoints":projallseason , "Avg": avg, "Last": last, "FFTeam": ffteam, "Week": week, "TimeUpdated": timeupdated},ignore_index=True)
        elif player == "Empty":
            pass
            play = "Empty"
            ac[1] = ""
            pos = None
    #bbdf = bbdf.append({'Slot': slot, 'Name': play, 'NFL_Team': team, 'Pos': pos,'Injury_Status': inj, 'Opponent':opp, "Kickoff": kickoff, 'Proj': proj, 'Score': score, 'OPRK':oprk, '%Start': percstart, "%Owned": percrost, "OwnerChange":ownchange, "PRK": prk, "EstPoints":projallseason , "Avg": avg, "Last": last, "FFTeam": ffteam, "Week": week, "TimeUpdated": timeupdated},ignore_index=True)
    #print(f'{slot}, {play}, {ac[1]}')
    
ssdf

,Slot,Name,NFL_Team,Pos,Injury_Status,Opponent,Kickoff,Proj,Score,OPRK,%Start,%Owned,OwnerChange,PRK,EstPoints,Avg,Last,FFTeam,Week,TimeUpdated
0,QB,Justin Herbert,LAC,QB,,LV,L 26-31,21.9,23.44,28th,71.1,96.5,+13.8,9,170.44,24.3,23.44,Team Sunshine,9,2020-11-10 10:00:01.708916
1,RB,Ezekiel Elliott,Dal,RB,,Pit,L 19-24,10.9,6.9,5th,43.6,99.9,0,6,111.0,12.3,6.9,Team Sunshine,9,2020-11-10 10:00:01.708916
2,RB,JaMycal Hasty,SF,RB,,GB,L 17-34,12.5,1.3,31st,38.7,80.8,+35.1,71,23.2,3.3,1.3,Team Sunshine,9,2020-11-10 10:00:01.708916
3,WR,Julio Jones,Atl,WR,,Den,W 34-27,12.2,11.4,24th,58.9,99.9,0,18,81.8,11.7,11.4,Team Sunshine,9,2020-11-10 10:00:01.708916
4,WR,DeAndre Hopkins,Ari,WR,,Mia,L 31-34,11.1,3.0,20th,96.8,100.0,0,9,89.4,11.2,3.0,Team Sunshine,9,2020-11-10 10:00:01.708916
5,TE,Hunter Henry,LAC,TE,,LV,L 26-31,7.2,3.3,12th,69.9,94.5,+3.8,18,41.7,5.2,3.3,Team Sunshine,9,2020-11-10 10:00:01.708916
6,FLEX,Brandin Cooks,Hou,WR,,@Jax,W 27-25,8.5,14.3,22nd,59.3,92.2,+14.7,28,69.0,8.6,14.3,Team Sunshine,9,2020-11-10 10:00:01.708916
7,D/ST,Ravens D/ST,Bal,D/ST,,@Ind,W 24-10,6.4,13.0,15th,88.2,98.4,+3.6,2,84.0,10.5,13.0,Team Sunshine,9,2020-11-10 10:00:01.708916
8,D/ST,Ravens D/SalD/ST,TB,D/ST,,@Ind,W 24-10,6.4,13.0,15th,88.2,98.4,+3.6,2,84.0,10.5,13.0,Team Sunshine,9,2020-11-10 10:00:01.708916
9,K,Greg Zuerlein,Dal,K,,Pit,L 19-24,7.1,15.0,6th,36.7,42.6,-10.2,13,74.0,8.2,15.0,Team Sunshine,9,2020-11-10 10:00:01.708916


In [28]:
ssdf.to_csv("C:\\Users\\mpden\\Documents\\PythonScripts\\FFL\\Data\\most_recent\\ss.csv")
print("Tony Collected")

Tony Collected


# Get Taylor

In [29]:
print('Collecting Taylor')
time.sleep(8)
search_box = driver.find_element_by_xpath("//li[@class = 'teams NavSecondary__Item']")
search_box.click()


##Selecting Fantasy League
time.sleep(2)
leaguego = driver.find_element_by_partial_link_text('Weenie')
leaguego.click()

# time.sleep(2)
# leaguego = driver.find_element_by_xpath("//span[@class='jsx-4290203343 week week-range']")
# leaguego.click()
print('Week 9')

# time.sleep(3)

Week 9


In [30]:
time.sleep(8)
page = driver.page_source
soup = BeautifulSoup(page, 'html.parser')
tables = soup.find_all("table")

table = tables[0]
tab_data = [[cell.text for cell in row.find_all(["th","td"])]
                        for row in table.find_all("tr")]
df = pd.DataFrame(tab_data)
gdf = df.drop([0,11,19])

gdf.columns = gdf.iloc[0]
gdf = gdf[1:]
#gdf.drop(columns=['action'])

gdf['Team'] = 'Weenie Hut Jrs'
gdf['Week'] = 9
gdf['Time'] = datetime.now()
gdf

1,SLOT,Player,opp,STATUS,proj,SCORE,OPRK,%ST,%ROST,+/-,PRK,FPTS,avg,LAST,Team,Week,Time
2,QB,Kyler MurrayAriQB,Mia,L 31-34,23.1,37.92,14th,91.3,99.9,+0.6,2,233.5,29.2,37.92,Weenie Hut Jrs,9,2020-11-10 10:01:56.822369
3,RB,Antonio GibsonWshRB,NYG,L 20-23,11.3,9.5,15th,79.6,97.8,+8.3,14,83.3,10.4,9.5,Weenie Hut Jrs,9,2020-11-10 10:01:56.822369
4,RB,James RobinsonJaxRB,Hou,L 25-27,14.7,15.9,30th,95.9,99.6,+4.7,4,122.5,15.3,15.9,Weenie Hut Jrs,9,2020-11-10 10:01:56.822369
5,WR,Davante AdamsGBWR,@SF,W 34-17,14.7,23.3,14th,98.8,100.0,0,3,115.5,19.3,23.3,Weenie Hut Jrs,9,2020-11-10 10:01:56.822369
6,WR,Tyler LockettSeaWR,@Buf,L 34-44,13.2,4.0,8th,96.1,99.9,+0.3,5,103.5,12.9,4.0,Weenie Hut Jrs,9,2020-11-10 10:01:56.822369
7,TE,Travis KelceKCTE,Car,W 33-31,11.3,15.9,13th,70.4,100.0,0,1,110.9,12.3,15.9,Weenie Hut Jrs,9,2020-11-10 10:01:56.822369
8,FLEX,Chase EdmondsAriRB,Mia,L 31-34,14.6,8.8,25th,75.3,98.1,+30.5,28,66.6,8.3,8.8,Weenie Hut Jrs,9,2020-11-10 10:01:56.822369
9,D/ST,Colts D/STIndD/ST,Bal,L 10-24,6.8,6.0,17th,77.5,94.9,+6.4,1,92.0,11.5,6.0,Weenie Hut Jrs,9,2020-11-10 10:01:56.822369
10,K,Harrison ButkerKCK,Car,W 33-31,8.9,10.0,25th,84.5,96.9,-1.3,10,77.0,8.6,10.0,Weenie Hut Jrs,9,2020-11-10 10:01:56.822369
12,Bench,Jonathan TaylorIndRB,Bal,L 10-24,10.2,7.4,1st,61.5,98.1,+1.3,17,81.4,10.2,7.4,Weenie Hut Jrs,9,2020-11-10 10:01:56.822369


In [31]:
nflteams = pd.read_csv('C:\\Users\\mpden\\Documents\\PythonScripts\\FFL\\Data\\static_tables\\teamacc.csv')
tcdf = pd.DataFrame(columns = ['Slot', 'Name', 'NFL_Team', 'Pos','Injury_Status', 'Opponent', 'Kickoff', 'Proj', 'Score', 'OPRK', '%Start', "%Owned", "OwnerChange", "PRK", "EstPoints", "Avg", "Last", "FFTeam", "Week", "TimeUpdated"])

for row,col in gdf.iterrows():
    slot = col[0]
    player = col[1]
    opp = col[2]
    kickoff = col[3]
    proj = col[4]
    score = col[5]
    oprk = col[6]
    percstart = col[7]
    percrost = col[8]
    ownchange = col[9]
    prk = col[10]
    projallseason = col[11]
    avg = col[12]
    last = col[13]
    ffteam = col[14]
    week = col[15]
    timeupdated = col[16]
    for team, ac in nflteams.iterrows():
        if len(ac[1])>2 and ("D/ST" not in player) and player[-1] != 'K' and ac[1] in player[-8:]:
            #Replace Team Acronym from Name String
            playpos = player.replace(ac[1], "")
            #Get to Injury status if aparent 
            play = playpos[-3]
            pos = playpos[-2:]
            if play[-1] =='Q':
                #print(f'{playpos[:-3]} and {ac[1]} and Questionable')
                play = playpos[:-3]
                inj = "Q"
                team = ac[1]
                tcdf = tcdf.append({'Slot': slot, 'Name': play, 'NFL_Team': team, 'Pos': pos,'Injury_Status': inj, 'Opponent':opp, "Kickoff": kickoff, 'Proj': proj, 'Score': score, 'OPRK':oprk, '%Start': percstart, "%Owned": percrost, "OwnerChange":ownchange, "PRK": prk, "EstPoints":projallseason , "Avg": avg, "Last": last, "FFTeam": ffteam, "Week": week, "TimeUpdated": timeupdated},ignore_index=True)
            else:
                #print(f'{playpos[:-2]} and {ac[1]}')
                play = playpos[:-2]
                inj = ""
                team = ac[1]
                tcdf = tcdf.append({'Slot': slot, 'Name': play, 'NFL_Team': team, 'Pos': pos,'Injury_Status': inj, 'Opponent':opp, "Kickoff": kickoff, 'Proj': proj, 'Score': score, 'OPRK':oprk, '%Start': percstart, "%Owned": percrost, "OwnerChange":ownchange, "PRK": prk, "EstPoints":projallseason , "Avg": avg, "Last": last, "FFTeam": ffteam, "Week": week, "TimeUpdated": timeupdated},ignore_index=True)
        elif len(ac[1])<3 and ("D/ST" not in player) and player[-1] != 'K' and ac[1] in player[-8:]:
            #Replace Team Acronym from Name String
            playpos = player.replace(ac[1], "")
            #Get to Injury status if aparent 
            play = playpos[:-3]
            pos = playpos[-2:]
            if play[-1] =='Q':
                #print(f'{playpos[:-3]} and {ac[1]} and Questionable')
                play = playpos[:-3]
                inj = "Q"
                team = ac[1]
                tcdf = tcdf.append({'Slot': slot, 'Name': play, 'NFL_Team': team, 'Pos': pos,'Injury_Status': inj, 'Opponent':opp, "Kickoff": kickoff, 'Proj': proj, 'Score': score, 'OPRK':oprk, '%Start': percstart, "%Owned": percrost, "OwnerChange":ownchange, "PRK": prk, "EstPoints":projallseason , "Avg": avg, "Last": last, "FFTeam": ffteam, "Week": week, "TimeUpdated": timeupdated},ignore_index=True)
            else:
                #print(f'{playpos[:-2]} and {ac[1]}')
                play = playpos[:-2]
                inj = ""
                team = ac[1]
                tcdf = tcdf.append({'Slot': slot, 'Name': play, 'NFL_Team': team, 'Pos': pos,'Injury_Status': inj, 'Opponent':opp, "Kickoff": kickoff, 'Proj': proj, 'Score': score, 'OPRK':oprk, '%Start': percstart, "%Owned": percrost, "OwnerChange":ownchange, "PRK": prk, "EstPoints":projallseason , "Avg": avg, "Last": last, "FFTeam": ffteam, "Week": week, "TimeUpdated": timeupdated},ignore_index=True)
        elif ac[1] in player[-8:] and player[-1] =="K":
            #Replace Team Acronym from Name String
            playpos = player.replace(ac[1], "")
            #Get to Injury status if aparent 
            pos = playpos[-2]
            pos = "K"
            if pos == "Q":
                #print("Questionable")
                inj = "Q"
                team = ac[1]
                tcdf = tcdf.append({'Slot': slot, 'Name': play, 'NFL_Team': team, 'Pos': pos,'Injury_Status': inj, 'Opponent':opp, "Kickoff": kickoff, 'Proj': proj, 'Score': score, 'OPRK':oprk, '%Start': percstart, "%Owned": percrost, "OwnerChange":ownchange, "PRK": prk, "EstPoints":projallseason , "Avg": avg, "Last": last, "FFTeam": ffteam, "Week": week, "TimeUpdated": timeupdated},ignore_index=True)
            else:
                #print(f'{playpos[:-1]} and {ac[1]}')
                play = playpos[:-1]
                inj = ""
                team = ac[1]
                tcdf = tcdf.append({'Slot': slot, 'Name': play, 'NFL_Team': team, 'Pos': pos,'Injury_Status': inj, 'Opponent':opp, "Kickoff": kickoff, 'Proj': proj, 'Score': score, 'OPRK':oprk, '%Start': percstart, "%Owned": percrost, "OwnerChange":ownchange, "PRK": prk, "EstPoints":projallseason , "Avg": avg, "Last": last, "FFTeam": ffteam, "Week": week, "TimeUpdated": timeupdated},ignore_index=True)
        elif ac[1] in player[-8:] and "D/ST" in player:
            #Replace Team Acronym from Name String
            playpos = player.replace(ac[1], "")
            play = playpos.replace(" D/ST", " ")
            pos = "D/ST"
            #print(play)
            team = ac[1]
            tcdf = tcdf.append({'Slot': slot, 'Name': play, 'NFL_Team': team, 'Pos': pos,'Injury_Status': inj, 'Opponent':opp, "Kickoff": kickoff, 'Proj': proj, 'Score': score, 'OPRK':oprk, '%Start': percstart, "%Owned": percrost, "OwnerChange":ownchange, "PRK": prk, "EstPoints":projallseason , "Avg": avg, "Last": last, "FFTeam": ffteam, "Week": week, "TimeUpdated": timeupdated},ignore_index=True)
        elif player == "Empty":
            pass
            play = "Empty"
            ac[1] = ""
            pos = None
    #bbdf = bbdf.append({'Slot': slot, 'Name': play, 'NFL_Team': team, 'Pos': pos,'Injury_Status': inj, 'Opponent':opp, "Kickoff": kickoff, 'Proj': proj, 'Score': score, 'OPRK':oprk, '%Start': percstart, "%Owned": percrost, "OwnerChange":ownchange, "PRK": prk, "EstPoints":projallseason , "Avg": avg, "Last": last, "FFTeam": ffteam, "Week": week, "TimeUpdated": timeupdated},ignore_index=True)
    #print(f'{slot}, {play}, {ac[1]}')
    
tcdf

,Slot,Name,NFL_Team,Pos,Injury_Status,Opponent,Kickoff,Proj,Score,OPRK,%Start,%Owned,OwnerChange,PRK,EstPoints,Avg,Last,FFTeam,Week,TimeUpdated
0,QB,Kyler Murray,Ari,QB,,Mia,L 31-34,23.1,37.92,14th,91.3,99.9,+0.6,2,233.5,29.2,37.92,Weenie Hut Jrs,9,2020-11-10 10:01:56.822369
1,RB,Antonio Gibson,Wsh,RB,,NYG,L 20-23,11.3,9.5,15th,79.6,97.8,+8.3,14,83.3,10.4,9.5,Weenie Hut Jrs,9,2020-11-10 10:01:56.822369
2,WR,Davante Adams,GB,WR,,@SF,W 34-17,14.7,23.3,14th,98.8,100.0,0,3,115.5,19.3,23.3,Weenie Hut Jrs,9,2020-11-10 10:01:56.822369
3,WR,Tyler Lockett,Sea,WR,,@Buf,L 34-44,13.2,4.0,8th,96.1,99.9,+0.3,5,103.5,12.9,4.0,Weenie Hut Jrs,9,2020-11-10 10:01:56.822369
4,TE,Travis Kelce,KC,TE,,Car,W 33-31,11.3,15.9,13th,70.4,100.0,0,1,110.9,12.3,15.9,Weenie Hut Jrs,9,2020-11-10 10:01:56.822369
5,FLEX,Chase Edmonds,Ari,RB,,Mia,L 31-34,14.6,8.8,25th,75.3,98.1,+30.5,28,66.6,8.3,8.8,Weenie Hut Jrs,9,2020-11-10 10:01:56.822369
6,D/ST,Colts D/ST,Ind,D/ST,,Bal,L 10-24,6.8,6.0,17th,77.5,94.9,+6.4,1,92.0,11.5,6.0,Weenie Hut Jrs,9,2020-11-10 10:01:56.822369
7,K,Harrison Butker,KC,K,,Car,W 33-31,8.9,10.0,25th,84.5,96.9,-1.3,10,77.0,8.6,10.0,Weenie Hut Jrs,9,2020-11-10 10:01:56.822369
8,Bench,Jonathan Taylor,Ind,RB,,Bal,L 10-24,10.2,7.4,1st,61.5,98.1,+1.3,17,81.4,10.2,7.4,Weenie Hut Jrs,9,2020-11-10 10:01:56.822369
9,Bench,Darius Slayton,NYG,WR,,@Wsh,W 23-20,7.6,0.8,1st,28.0,76.6,+9,34,65.0,7.2,0.8,Weenie Hut Jrs,9,2020-11-10 10:01:56.822369


In [32]:
tcdf.to_csv("C:\\Users\\mpden\\Documents\\PythonScripts\\FFL\\Data\\most_recent\\tc.csv")
print("Taylor Collected")

Taylor Collected


# Get TK

In [33]:
print("Collecting TK")
time.sleep(8)
search_box = driver.find_element_by_xpath("//li[@class = 'teams NavSecondary__Item']")
search_box.click()


##Selecting Fantasy League
time.sleep(2)
leaguego = driver.find_element_by_partial_link_text('Cash')
leaguego.click()

# time.sleep(2)
# leaguego = driver.find_element_by_xpath("//span[@class='jsx-4290203343 week week-range']")
# leaguego.click()
print('Week 9')

# time.sleep(3)

Week 9


In [34]:
time.sleep(8)
page = driver.page_source
soup = BeautifulSoup(page, 'html.parser')
tables = soup.find_all("table")

table = tables[0]
tab_data = [[cell.text for cell in row.find_all(["th","td"])]
                        for row in table.find_all("tr")]
df = pd.DataFrame(tab_data)
gdf = df.drop([0,11,19])

gdf.columns = gdf.iloc[0]
gdf = gdf[1:]
#gdf.drop(columns=['action'])

gdf['Team'] = 'Straight Cash Homie'
gdf['Week'] = 9
gdf['Time'] = datetime.now()
gdf

1,SLOT,Player,opp,STATUS,proj,SCORE,OPRK,%ST,%ROST,+/-,PRK,FPTS,avg,LAST,Team,Week,Time
2,QB,Josh AllenBufQB,Sea,W 44-34,24.9,36.0,32nd,84.6,99.5,+1.1,4,215.58,24.0,36.0,Straight Cash Homie,9,2020-11-10 10:03:10.079883
3,RB,Clyde Edwards-HelaireKCRB,Car,W 33-31,11.1,9.4,27th,59.4,99.7,0,10,99.0,11.0,9.4,Straight Cash Homie,9,2020-11-10 10:03:10.079883
4,RB,James ConnerPitRB,@Dal,W 24-19,17.2,2.0,21st,96.7,99.9,+1.2,11,94.3,11.8,2.0,Straight Cash Homie,9,2020-11-10 10:03:10.079883
5,WR,Adam ThielenMinWR,Det,W 34-20,10.8,3.8,12th,89.6,99.8,+0.2,8,93.5,11.7,3.8,Straight Cash Homie,9,2020-11-10 10:03:10.079883
6,WR,Justin JeffersonMinWR,Det,W 34-20,9.1,6.4,12th,63.2,96.9,+7.8,16,82.7,10.3,6.4,Straight Cash Homie,9,2020-11-10 10:03:10.079883
7,TE,Mark AndrewsBalTE,@Ind,W 24-10,7.8,2.2,1st,85.5,99.6,+0.4,8,59.7,7.5,2.2,Straight Cash Homie,9,2020-11-10 10:03:10.079883
8,FLEX,Josh JacobsLVRB,@LAC,W 31-26,14.7,12.8,17th,96.4,99.9,+0.2,8,105.3,13.2,12.8,Straight Cash Homie,9,2020-11-10 10:03:10.079883
9,D/ST,Steelers D/STPitD/ST,@Dal,W 24-19,7.2,5.0,27th,96.3,99.7,+3.2,3,81.0,10.1,5.0,Straight Cash Homie,9,2020-11-10 10:03:10.079883
10,K,Daniel CarlsonLVK,@LAC,W 31-26,8.7,7.0,23rd,68.1,72.4,+37.7,7,80.0,10.0,7.0,Straight Cash Homie,9,2020-11-10 10:03:10.079883
12,Bench,Terry McLaurinWshWR,NYG,L 20-23,10.0,17.5,21st,87.3,99.2,+4.7,12,87.5,10.9,17.5,Straight Cash Homie,9,2020-11-10 10:03:10.079883


In [35]:
nflteams = pd.read_csv('C:\\Users\\mpden\\Documents\\PythonScripts\\FFL\\Data\\static_tables\\teamacc.csv')
tkdf = pd.DataFrame(columns = ['Slot', 'Name', 'NFL_Team', 'Pos','Injury_Status', 'Opponent', 'Kickoff', 'Proj', 'Score', 'OPRK', '%Start', "%Owned", "OwnerChange", "PRK", "EstPoints", "Avg", "Last", "FFTeam", "Week", "TimeUpdated"])

for row,col in gdf.iterrows():
    slot = col[0]
    player = col[1]
    opp = col[2]
    kickoff = col[3]
    proj = col[4]
    score = col[5]
    oprk = col[6]
    percstart = col[7]
    percrost = col[8]
    ownchange = col[9]
    prk = col[10]
    projallseason = col[11]
    avg = col[12]
    last = col[13]
    ffteam = col[14]
    week = col[15]
    timeupdated = col[16]
    for team, ac in nflteams.iterrows():
        if len(ac[1])>2 and ("D/ST" not in player) and player[-1] != 'K' and ac[1] in player[-8:]:
            #Replace Team Acronym from Name String
            playpos = player.replace(ac[1], "")
            #Get to Injury status if aparent 
            play = playpos[-3]
            pos = playpos[-2:]
            if play[-1] =='Q':
                #print(f'{playpos[:-3]} and {ac[1]} and Questionable')
                play = playpos[:-3]
                inj = "Q"
                team = ac[1]
                tkdf = tkdf.append({'Slot': slot, 'Name': play, 'NFL_Team': team, 'Pos': pos,'Injury_Status': inj, 'Opponent':opp, "Kickoff": kickoff, 'Proj': proj, 'Score': score, 'OPRK':oprk, '%Start': percstart, "%Owned": percrost, "OwnerChange":ownchange, "PRK": prk, "EstPoints":projallseason , "Avg": avg, "Last": last, "FFTeam": ffteam, "Week": week, "TimeUpdated": timeupdated},ignore_index=True)
            else:
                #print(f'{playpos[:-2]} and {ac[1]}')
                play = playpos[:-2]
                inj = ""
                team = ac[1]
                tkdf = tkdf.append({'Slot': slot, 'Name': play, 'NFL_Team': team, 'Pos': pos,'Injury_Status': inj, 'Opponent':opp, "Kickoff": kickoff, 'Proj': proj, 'Score': score, 'OPRK':oprk, '%Start': percstart, "%Owned": percrost, "OwnerChange":ownchange, "PRK": prk, "EstPoints":projallseason , "Avg": avg, "Last": last, "FFTeam": ffteam, "Week": week, "TimeUpdated": timeupdated},ignore_index=True)
        elif len(ac[1])<3 and ("D/ST" not in player) and player[-1] != 'K' and ac[1] in player[-8:]:
            #Replace Team Acronym from Name String
            playpos = player.replace(ac[1], "")
            #Get to Injury status if aparent 
            play = playpos[:-3]
            pos = playpos[-2:]
            if play[-1] =='Q':
                #print(f'{playpos[:-3]} and {ac[1]} and Questionable')
                play = playpos[:-3]
                inj = "Q"
                team = ac[1]
                tkdf = tkdf.append({'Slot': slot, 'Name': play, 'NFL_Team': team, 'Pos': pos,'Injury_Status': inj, 'Opponent':opp, "Kickoff": kickoff, 'Proj': proj, 'Score': score, 'OPRK':oprk, '%Start': percstart, "%Owned": percrost, "OwnerChange":ownchange, "PRK": prk, "EstPoints":projallseason , "Avg": avg, "Last": last, "FFTeam": ffteam, "Week": week, "TimeUpdated": timeupdated},ignore_index=True)
            else:
                #print(f'{playpos[:-2]} and {ac[1]}')
                play = playpos[:-2]
                inj = ""
                team = ac[1]
                tkdf = tkdf.append({'Slot': slot, 'Name': play, 'NFL_Team': team, 'Pos': pos,'Injury_Status': inj, 'Opponent':opp, "Kickoff": kickoff, 'Proj': proj, 'Score': score, 'OPRK':oprk, '%Start': percstart, "%Owned": percrost, "OwnerChange":ownchange, "PRK": prk, "EstPoints":projallseason , "Avg": avg, "Last": last, "FFTeam": ffteam, "Week": week, "TimeUpdated": timeupdated},ignore_index=True)
        elif ac[1] in player[-8:] and player[-1] =="K":
            #Replace Team Acronym from Name String
            playpos = player.replace(ac[1], "")
            #Get to Injury status if aparent 
            pos = playpos[-2]
            pos = "K"
            if pos == "Q":
                #print("Questionable")
                inj = "Q"
                team = ac[1]
                tkdf = tkdf.append({'Slot': slot, 'Name': play, 'NFL_Team': team, 'Pos': pos,'Injury_Status': inj, 'Opponent':opp, "Kickoff": kickoff, 'Proj': proj, 'Score': score, 'OPRK':oprk, '%Start': percstart, "%Owned": percrost, "OwnerChange":ownchange, "PRK": prk, "EstPoints":projallseason , "Avg": avg, "Last": last, "FFTeam": ffteam, "Week": week, "TimeUpdated": timeupdated},ignore_index=True)
            else:
                #print(f'{playpos[:-1]} and {ac[1]}')
                play = playpos[:-1]
                inj = ""
                team = ac[1]
                tkdf = tkdf.append({'Slot': slot, 'Name': play, 'NFL_Team': team, 'Pos': pos,'Injury_Status': inj, 'Opponent':opp, "Kickoff": kickoff, 'Proj': proj, 'Score': score, 'OPRK':oprk, '%Start': percstart, "%Owned": percrost, "OwnerChange":ownchange, "PRK": prk, "EstPoints":projallseason , "Avg": avg, "Last": last, "FFTeam": ffteam, "Week": week, "TimeUpdated": timeupdated},ignore_index=True)
        elif ac[1] in player[-8:] and "D/ST" in player:
            #Replace Team Acronym from Name String
            playpos = player.replace(ac[1], "")
            play = playpos.replace(" D/ST", " ")
            pos = "D/ST"
            #print(play)
            team = ac[1]
            tkdf = tkdf.append({'Slot': slot, 'Name': play, 'NFL_Team': team, 'Pos': pos,'Injury_Status': inj, 'Opponent':opp, "Kickoff": kickoff, 'Proj': proj, 'Score': score, 'OPRK':oprk, '%Start': percstart, "%Owned": percrost, "OwnerChange":ownchange, "PRK": prk, "EstPoints":projallseason , "Avg": avg, "Last": last, "FFTeam": ffteam, "Week": week, "TimeUpdated": timeupdated},ignore_index=True)
        elif player == "Empty":
            pass
            play = "Empty"
            ac[1] = ""
            pos = None
    #bbdf = bbdf.append({'Slot': slot, 'Name': play, 'NFL_Team': team, 'Pos': pos,'Injury_Status': inj, 'Opponent':opp, "Kickoff": kickoff, 'Proj': proj, 'Score': score, 'OPRK':oprk, '%Start': percstart, "%Owned": percrost, "OwnerChange":ownchange, "PRK": prk, "EstPoints":projallseason , "Avg": avg, "Last": last, "FFTeam": ffteam, "Week": week, "TimeUpdated": timeupdated},ignore_index=True)
    #print(f'{slot}, {play}, {ac[1]}')
    
tkdf

,Slot,Name,NFL_Team,Pos,Injury_Status,Opponent,Kickoff,Proj,Score,OPRK,%Start,%Owned,OwnerChange,PRK,EstPoints,Avg,Last,FFTeam,Week,TimeUpdated
0,QB,Josh Allen,Buf,QB,,Sea,W 44-34,24.9,36.0,32nd,84.6,99.5,+1.1,4,215.58,24.0,36.0,Straight Cash Homie,9,2020-11-10 10:03:10.079883
1,RB,Clyde Edwards-Helaire,KC,RB,,Car,W 33-31,11.1,9.4,27th,59.4,99.7,0,10,99.0,11.0,9.4,Straight Cash Homie,9,2020-11-10 10:03:10.079883
2,RB,James Conner,Pit,RB,,@Dal,W 24-19,17.2,2.0,21st,96.7,99.9,+1.2,11,94.3,11.8,2.0,Straight Cash Homie,9,2020-11-10 10:03:10.079883
3,WR,Adam Thielen,Min,WR,,Det,W 34-20,10.8,3.8,12th,89.6,99.8,+0.2,8,93.5,11.7,3.8,Straight Cash Homie,9,2020-11-10 10:03:10.079883
4,WR,Justin Jefferson,Min,WR,,Det,W 34-20,9.1,6.4,12th,63.2,96.9,+7.8,16,82.7,10.3,6.4,Straight Cash Homie,9,2020-11-10 10:03:10.079883
5,TE,Mark Andrews,Bal,TE,,@Ind,W 24-10,7.8,2.2,1st,85.5,99.6,+0.4,8,59.7,7.5,2.2,Straight Cash Homie,9,2020-11-10 10:03:10.079883
6,FLEX,Josh Jacobs,LV,RB,,@LAC,W 31-26,14.7,12.8,17th,96.4,99.9,+0.2,8,105.3,13.2,12.8,Straight Cash Homie,9,2020-11-10 10:03:10.079883
7,D/ST,Steelers D/ST,Pit,D/ST,,@Dal,W 24-19,7.2,5.0,27th,96.3,99.7,+3.2,3,81.0,10.1,5.0,Straight Cash Homie,9,2020-11-10 10:03:10.079883
8,K,Daniel Carlson,LV,K,,@LAC,W 31-26,8.7,7.0,23rd,68.1,72.4,+37.7,7,80.0,10.0,7.0,Straight Cash Homie,9,2020-11-10 10:03:10.079883
9,Bench,Terry McLaurin,Wsh,WR,,NYG,L 20-23,10.0,17.5,21st,87.3,99.2,+4.7,12,87.5,10.9,17.5,Straight Cash Homie,9,2020-11-10 10:03:10.079883


In [36]:
tkdf.to_csv("C:\\Users\\mpden\\Documents\\PythonScripts\\FFL\\Data\\most_recent\\tk.csv")
print("TK Collected")

TK Collected


# Get Jerry

In [37]:
print("Collecting Jerry")
time.sleep(8)
search_box = driver.find_element_by_xpath("//li[@class = 'teams NavSecondary__Item']")
search_box.click()


##Selecting Fantasy League
time.sleep(2)
leaguego = driver.find_element_by_partial_link_text('Jerem')
leaguego.click()

# time.sleep(2)
# leaguego = driver.find_element_by_xpath("//span[@class='jsx-4290203343 week week-range']")
# leaguego.click()
print('Week 9')

# time.sleep(3)

Week 9


In [38]:
time.sleep(8)
page = driver.page_source
soup = BeautifulSoup(page, 'html.parser')
tables = soup.find_all("table")

table = tables[0]
tab_data = [[cell.text for cell in row.find_all(["th","td"])]
                        for row in table.find_all("tr")]
df = pd.DataFrame(tab_data)
gdf = df.drop([0,11,19])

gdf.columns = gdf.iloc[0]
gdf = gdf[1:]
#gdf.drop(columns=['action'])

gdf['Team'] = 'Two Gurleys One Kupp'
gdf['Week'] = 9
gdf['Time'] = datetime.now()
gdf

1,SLOT,Player,opp,STATUS,proj,SCORE,OPRK,%ST,%ROST,+/-,PRK,FPTS,avg,LAST,Team,Week,Time
2,QB,Deshaun WatsonHouQB,@Jax,W 27-25,21.9,24.24,30th,81.4,99.1,+1.3,6,178.74,22.3,24.24,Two Gurleys One Kupp,9,2020-11-10 10:04:00.731778
3,RB,Dalvin CookMinRB,Det,W 34-20,20.4,37.2,32nd,98.3,100.0,0,1,185.1,26.4,37.2,Two Gurleys One Kupp,9,2020-11-10 10:04:00.731778
4,RB,Leonard FournetteTBRB,NO,L 3-38,11.4,4.1,2nd,69.2,96.0,+9.1,43,48.6,6.9,4.1,Two Gurleys One Kupp,9,2020-11-10 10:04:00.731778
5,WR,Stefon DiggsBufWR,Sea,W 44-34,12.9,11.8,32nd,97.0,99.9,+0.8,6,99.3,11.0,11.8,Two Gurleys One Kupp,9,2020-11-10 10:04:00.731778
6,WR,Marvin Jones Jr.DetWR,@Min,L 20-34,9.7,10.3,31st,53.1,91.2,+37,46,54.8,6.9,10.3,Two Gurleys One Kupp,9,2020-11-10 10:04:00.731778
7,TE,Eric EbronPitTE,@Dal,W 24-19,6.4,8.2,18th,56.3,80.0,+28.8,16,44.5,5.6,8.2,Two Gurleys One Kupp,9,2020-11-10 10:04:00.731778
8,FLEX,Jamison CrowderNYJWR,NE,L 27-30,8.4,8.6,16th,20.6,92.0,+7.9,53,58.9,11.8,8.6,Two Gurleys One Kupp,9,2020-11-10 10:04:00.731778
9,D/ST,49ers D/STSFD/ST,GB,L 17-34,4.1,-3.0,1st,35.0,58.9,-12.8,14,39.0,4.3,-3.0,Two Gurleys One Kupp,9,2020-11-10 10:04:00.731778
10,K,Matt PraterDetK,@Min,L 20-34,8.4,8.0,32nd,59.0,66.2,-1.1,18,65.0,8.1,8.0,Two Gurleys One Kupp,9,2020-11-10 10:04:00.731778
12,Bench,Kenyan DrakeQAriRB,Mia,L 31-34,0.0,--,25th,18.8,91.5,+1.3,22,78.1,11.2,--,Two Gurleys One Kupp,9,2020-11-10 10:04:00.731778


In [39]:
nflteams = pd.read_csv('C:\\Users\\mpden\\Documents\\PythonScripts\\FFL\\Data\\static_tables\\teamacc.csv')
jjdf = pd.DataFrame(columns = ['Slot', 'Name', 'NFL_Team', 'Pos','Injury_Status', 'Opponent', 'Kickoff', 'Proj', 'Score', 'OPRK', '%Start', "%Owned", "OwnerChange", "PRK", "EstPoints", "Avg", "Last", "FFTeam", "Week", "TimeUpdated"])

for row,col in gdf.iterrows():
    slot = col[0]
    player = col[1]
    opp = col[2]
    kickoff = col[3]
    proj = col[4]
    score = col[5]
    oprk = col[6]
    percstart = col[7]
    percrost = col[8]
    ownchange = col[9]
    prk = col[10]
    projallseason = col[11]
    avg = col[12]
    last = col[13]
    ffteam = col[14]
    week = col[15]
    timeupdated = col[16]
    for team, ac in nflteams.iterrows():
        if len(ac[1])>2 and ("D/ST" not in player) and player[-1] != 'K' and ac[1] in player[-8:]:
            #Replace Team Acronym from Name String
            playpos = player.replace(ac[1], "")
            #Get to Injury status if aparent 
            play = playpos[-3]
            pos = playpos[-2:]
            if play[-1] =='Q':
                #print(f'{playpos[:-3]} and {ac[1]} and Questionable')
                play = playpos[:-3]
                inj = "Q"
                team = ac[1]
                jjdf = jjdf.append({'Slot': slot, 'Name': play, 'NFL_Team': team, 'Pos': pos,'Injury_Status': inj, 'Opponent':opp, "Kickoff": kickoff, 'Proj': proj, 'Score': score, 'OPRK':oprk, '%Start': percstart, "%Owned": percrost, "OwnerChange":ownchange, "PRK": prk, "EstPoints":projallseason , "Avg": avg, "Last": last, "FFTeam": ffteam, "Week": week, "TimeUpdated": timeupdated},ignore_index=True)
            else:
                #print(f'{playpos[:-2]} and {ac[1]}')
                play = playpos[:-2]
                inj = ""
                team = ac[1]
                jjdf = jjdf.append({'Slot': slot, 'Name': play, 'NFL_Team': team, 'Pos': pos,'Injury_Status': inj, 'Opponent':opp, "Kickoff": kickoff, 'Proj': proj, 'Score': score, 'OPRK':oprk, '%Start': percstart, "%Owned": percrost, "OwnerChange":ownchange, "PRK": prk, "EstPoints":projallseason , "Avg": avg, "Last": last, "FFTeam": ffteam, "Week": week, "TimeUpdated": timeupdated},ignore_index=True)
        elif len(ac[1])<3 and ("D/ST" not in player) and player[-1] != 'K' and ac[1] in player[-8:]:
            #Replace Team Acronym from Name String
            playpos = player.replace(ac[1], "")
            #Get to Injury status if aparent 
            play = playpos[:-3]
            pos = playpos[-2:]
            if play[-1] =='Q':
                #print(f'{playpos[:-3]} and {ac[1]} and Questionable')
                play = playpos[:-3]
                inj = "Q"
                team = ac[1]
                jjdf = jjdf.append({'Slot': slot, 'Name': play, 'NFL_Team': team, 'Pos': pos,'Injury_Status': inj, 'Opponent':opp, "Kickoff": kickoff, 'Proj': proj, 'Score': score, 'OPRK':oprk, '%Start': percstart, "%Owned": percrost, "OwnerChange":ownchange, "PRK": prk, "EstPoints":projallseason , "Avg": avg, "Last": last, "FFTeam": ffteam, "Week": week, "TimeUpdated": timeupdated},ignore_index=True)
            else:
                #print(f'{playpos[:-2]} and {ac[1]}')
                play = playpos[:-2]
                inj = ""
                team = ac[1]
                jjdf = jjdf.append({'Slot': slot, 'Name': play, 'NFL_Team': team, 'Pos': pos,'Injury_Status': inj, 'Opponent':opp, "Kickoff": kickoff, 'Proj': proj, 'Score': score, 'OPRK':oprk, '%Start': percstart, "%Owned": percrost, "OwnerChange":ownchange, "PRK": prk, "EstPoints":projallseason , "Avg": avg, "Last": last, "FFTeam": ffteam, "Week": week, "TimeUpdated": timeupdated},ignore_index=True)
        elif ac[1] in player[-8:] and player[-1] =="K":
            #Replace Team Acronym from Name String
            playpos = player.replace(ac[1], "")
            #Get to Injury status if aparent 
            pos = playpos[-2]
            pos = "K"
            if pos == "Q":
                #print("Questionable")
                inj = "Q"
                team = ac[1]
                jjdf = jjdf.append({'Slot': slot, 'Name': play, 'NFL_Team': team, 'Pos': pos,'Injury_Status': inj, 'Opponent':opp, "Kickoff": kickoff, 'Proj': proj, 'Score': score, 'OPRK':oprk, '%Start': percstart, "%Owned": percrost, "OwnerChange":ownchange, "PRK": prk, "EstPoints":projallseason , "Avg": avg, "Last": last, "FFTeam": ffteam, "Week": week, "TimeUpdated": timeupdated},ignore_index=True)
            else:
                #print(f'{playpos[:-1]} and {ac[1]}')
                play = playpos[:-1]
                inj = ""
                team = ac[1]
                jjdf = jjdf.append({'Slot': slot, 'Name': play, 'NFL_Team': team, 'Pos': pos,'Injury_Status': inj, 'Opponent':opp, "Kickoff": kickoff, 'Proj': proj, 'Score': score, 'OPRK':oprk, '%Start': percstart, "%Owned": percrost, "OwnerChange":ownchange, "PRK": prk, "EstPoints":projallseason , "Avg": avg, "Last": last, "FFTeam": ffteam, "Week": week, "TimeUpdated": timeupdated},ignore_index=True)
        elif ac[1] in player[-8:] and "D/ST" in player:
            #Replace Team Acronym from Name String
            playpos = player.replace(ac[1], "")
            play = playpos.replace(" D/ST", " ")
            pos = "D/ST"
            #print(play)
            team = ac[1]
            jjdf = jjdf.append({'Slot': slot, 'Name': play, 'NFL_Team': team, 'Pos': pos,'Injury_Status': inj, 'Opponent':opp, "Kickoff": kickoff, 'Proj': proj, 'Score': score, 'OPRK':oprk, '%Start': percstart, "%Owned": percrost, "OwnerChange":ownchange, "PRK": prk, "EstPoints":projallseason , "Avg": avg, "Last": last, "FFTeam": ffteam, "Week": week, "TimeUpdated": timeupdated},ignore_index=True)
        elif player == "Empty":
            pass
            play = "Empty"
            ac[1] = ""
            pos = None
    #bbdf = bbdf.append({'Slot': slot, 'Name': play, 'NFL_Team': team, 'Pos': pos,'Injury_Status': inj, 'Opponent':opp, "Kickoff": kickoff, 'Proj': proj, 'Score': score, 'OPRK':oprk, '%Start': percstart, "%Owned": percrost, "OwnerChange":ownchange, "PRK": prk, "EstPoints":projallseason , "Avg": avg, "Last": last, "FFTeam": ffteam, "Week": week, "TimeUpdated": timeupdated},ignore_index=True)
    #print(f'{slot}, {play}, {ac[1]}')
    
jjdf

,Slot,Name,NFL_Team,Pos,Injury_Status,Opponent,Kickoff,Proj,Score,OPRK,%Start,%Owned,OwnerChange,PRK,EstPoints,Avg,Last,FFTeam,Week,TimeUpdated
0,QB,Deshaun Watson,Hou,QB,,@Jax,W 27-25,21.9,24.24,30th,81.4,99.1,+1.3,6,178.74,22.3,24.24,Two Gurleys One Kupp,9,2020-11-10 10:04:00.731778
1,RB,Dalvin Cook,Min,RB,,Det,W 34-20,20.4,37.2,32nd,98.3,100.0,0,1,185.1,26.4,37.2,Two Gurleys One Kupp,9,2020-11-10 10:04:00.731778
2,RB,Leonard Fournette,TB,RB,,NO,L 3-38,11.4,4.1,2nd,69.2,96.0,+9.1,43,48.6,6.9,4.1,Two Gurleys One Kupp,9,2020-11-10 10:04:00.731778
3,WR,Stefon Diggs,Buf,WR,,Sea,W 44-34,12.9,11.8,32nd,97.0,99.9,+0.8,6,99.3,11.0,11.8,Two Gurleys One Kupp,9,2020-11-10 10:04:00.731778
4,WR,Marvin Jones Jr.,Det,WR,,@Min,L 20-34,9.7,10.3,31st,53.1,91.2,+37,46,54.8,6.9,10.3,Two Gurleys One Kupp,9,2020-11-10 10:04:00.731778
5,TE,Eric Ebron,Pit,TE,,@Dal,W 24-19,6.4,8.2,18th,56.3,80.0,+28.8,16,44.5,5.6,8.2,Two Gurleys One Kupp,9,2020-11-10 10:04:00.731778
6,FLEX,Jamison Crowder,NYJ,WR,,NE,L 27-30,8.4,8.6,16th,20.6,92.0,+7.9,53,58.9,11.8,8.6,Two Gurleys One Kupp,9,2020-11-10 10:04:00.731778
7,D/ST,49ers D/ST,SF,D/ST,,GB,L 17-34,4.1,-3.0,1st,35.0,58.9,-12.8,14,39.0,4.3,-3.0,Two Gurleys One Kupp,9,2020-11-10 10:04:00.731778
8,K,Matt Prater,Det,K,,@Min,L 20-34,8.4,8.0,32nd,59.0,66.2,-1.1,18,65.0,8.1,8.0,Two Gurleys One Kupp,9,2020-11-10 10:04:00.731778
9,Bench,Kenyan Drake,Ari,RB,Q,Mia,L 31-34,0.0,--,25th,18.8,91.5,+1.3,22,78.1,11.2,--,Two Gurleys One Kupp,9,2020-11-10 10:04:00.731778


In [40]:
jjdf.to_csv("C:\\Users\\mpden\\Documents\\PythonScripts\\FFL\\Data\\most_recent\\jj.csv")
print("Jerry Collected")

Jerry Collected


# Get Oberle

In [41]:
print("Collecting Oberle")
time.sleep(8)
search_box = driver.find_element_by_xpath("//li[@class = 'teams NavSecondary__Item']")
search_box.click()


##Selecting Fantasy League
time.sleep(2)
leaguego = driver.find_element_by_partial_link_text('Prostate')
leaguego.click()

# time.sleep(2)
# leaguego = driver.find_element_by_xpath("//span[@class='jsx-4290203343 week week-range']")
# leaguego.click()
print('Week 9')

# time.sleep(3)

Week 8


In [42]:
time.sleep(8)
page = driver.page_source
soup = BeautifulSoup(page, 'html.parser')
tables = soup.find_all("table")

table = tables[0]
tab_data = [[cell.text for cell in row.find_all(["th","td"])]
                        for row in table.find_all("tr")]
df = pd.DataFrame(tab_data)
gdf = df.drop([0,11,19])

gdf.columns = gdf.iloc[0]
gdf = gdf[1:]
#gdf.drop(columns=['action'])

gdf['Team'] = 'Prostate Prophet'
gdf['Week'] = 9
gdf['Time'] = datetime.now()
gdf

1,SLOT,Player,opp,STATUS,proj,SCORE,OPRK,%ST,%ROST,+/-,PRK,FPTS,avg,LAST,Team,Week,Time
2,QB,Patrick MahomesKCQB,Car,W 33-31,23.0,30.88,10th,68.9,100.0,0,1,237.98,26.4,30.88,Prostate Prophet,9,2020-11-10 10:04:56.171547
3,RB,Alvin KamaraNORB,@TB,W 38-3,14.7,10.9,9th,99.3,100.0,0,2,151.6,19.0,10.9,Prostate Prophet,9,2020-11-10 10:04:56.171547
4,RB,D'Andre SwiftDetRB,@Min,L 20-34,10.6,9.7,16th,69.3,95.8,+7.4,23,75.7,9.5,9.7,Prostate Prophet,9,2020-11-10 10:04:56.171547
5,WR,Tyreek HillKCWR,Car,W 33-31,11.5,24.1,6th,59.1,99.9,0,1,131.2,14.6,24.1,Prostate Prophet,9,2020-11-10 10:04:56.171547
6,WR,Robby AndersonCarWR,@KC,L 31-33,8.9,6.3,4th,77.7,98.8,+3.8,20,81.4,9.0,6.3,Prostate Prophet,9,2020-11-10 10:04:56.171547
7,TE,Noah FantQDenTE,@Atl,L 27-34,7.1,4.5,32nd,72.4,94.2,+13.5,12,48.9,7.0,4.5,Prostate Prophet,9,2020-11-10 10:04:56.171547
8,FLEX,Jerick McKinnonSFRB,GB,L 17-34,9.5,12.8,31st,47.8,86.9,+14.6,19,79.4,8.8,12.8,Prostate Prophet,9,2020-11-10 10:04:56.171547
9,D/ST,Cardinals D/STAriD/ST,Mia,L 31-34,7.2,2.0,13th,54.9,68.4,+53.2,14,39.0,4.9,2.0,Prostate Prophet,9,2020-11-10 10:04:56.171547
10,K,Ryan SuccopTBK,NO,L 3-38,8.7,4.0,12th,61.1,65.2,+44.5,9,79.0,8.8,4.0,Prostate Prophet,9,2020-11-10 10:04:56.171547
12,Bench,Kenny GolladayQDetWR,@Min,L 20-34,0.0,--,31st,23.3,97.1,-0.6,59,45.8,9.2,--,Prostate Prophet,9,2020-11-10 10:04:56.171547


In [43]:
nflteams = pd.read_csv('C:\\Users\\mpden\\Documents\\PythonScripts\\FFL\\Data\\static_tables\\teamacc.csv')
obdf = pd.DataFrame(columns = ['Slot', 'Name', 'NFL_Team', 'Pos','Injury_Status', 'Opponent', 'Kickoff', 'Proj', 'Score', 'OPRK', '%Start', "%Owned", "OwnerChange", "PRK", "EstPoints", "Avg", "Last", "FFTeam", "Week", "TimeUpdated"])

for row,col in gdf.iterrows():
    slot = col[0]
    player = col[1]
    opp = col[2]
    kickoff = col[3]
    proj = col[4]
    score = col[5]
    oprk = col[6]
    percstart = col[7]
    percrost = col[8]
    ownchange = col[9]
    prk = col[10]
    projallseason = col[11]
    avg = col[12]
    last = col[13]
    ffteam = col[14]
    week = col[15]
    timeupdated = col[16]
    for team, ac in nflteams.iterrows():
        if len(ac[1])>2 and ("D/ST" not in player) and player[-1] != 'K' and ac[1] in player[-8:]:
            #Replace Team Acronym from Name String
            playpos = player.replace(ac[1], "")
            #Get to Injury status if aparent 
            play = playpos[-3]
            pos = playpos[-2:]
            if play[-1] =='Q':
                #print(f'{playpos[:-3]} and {ac[1]} and Questionable')
                play = playpos[:-3]
                inj = "Q"
                team = ac[1]
                obdf = obdf.append({'Slot': slot, 'Name': play, 'NFL_Team': team, 'Pos': pos,'Injury_Status': inj, 'Opponent':opp, "Kickoff": kickoff, 'Proj': proj, 'Score': score, 'OPRK':oprk, '%Start': percstart, "%Owned": percrost, "OwnerChange":ownchange, "PRK": prk, "EstPoints":projallseason , "Avg": avg, "Last": last, "FFTeam": ffteam, "Week": week, "TimeUpdated": timeupdated},ignore_index=True)
            else:
                #print(f'{playpos[:-2]} and {ac[1]}')
                play = playpos[:-2]
                inj = ""
                team = ac[1]
                obdf = obdf.append({'Slot': slot, 'Name': play, 'NFL_Team': team, 'Pos': pos,'Injury_Status': inj, 'Opponent':opp, "Kickoff": kickoff, 'Proj': proj, 'Score': score, 'OPRK':oprk, '%Start': percstart, "%Owned": percrost, "OwnerChange":ownchange, "PRK": prk, "EstPoints":projallseason , "Avg": avg, "Last": last, "FFTeam": ffteam, "Week": week, "TimeUpdated": timeupdated},ignore_index=True)
        elif len(ac[1])<3 and ("D/ST" not in player) and player[-1] != 'K' and ac[1] in player[-8:]:
            #Replace Team Acronym from Name String
            playpos = player.replace(ac[1], "")
            #Get to Injury status if aparent 
            play = playpos[:-3]
            pos = playpos[-2:]
            if play[-1] =='Q':
                #print(f'{playpos[:-3]} and {ac[1]} and Questionable')
                play = playpos[:-3]
                inj = "Q"
                team = ac[1]
                obdf = obdf.append({'Slot': slot, 'Name': play, 'NFL_Team': team, 'Pos': pos,'Injury_Status': inj, 'Opponent':opp, "Kickoff": kickoff, 'Proj': proj, 'Score': score, 'OPRK':oprk, '%Start': percstart, "%Owned": percrost, "OwnerChange":ownchange, "PRK": prk, "EstPoints":projallseason , "Avg": avg, "Last": last, "FFTeam": ffteam, "Week": week, "TimeUpdated": timeupdated},ignore_index=True)
            else:
                #print(f'{playpos[:-2]} and {ac[1]}')
                play = playpos[:-2]
                inj = ""
                team = ac[1]
                obdf = obdf.append({'Slot': slot, 'Name': play, 'NFL_Team': team, 'Pos': pos,'Injury_Status': inj, 'Opponent':opp, "Kickoff": kickoff, 'Proj': proj, 'Score': score, 'OPRK':oprk, '%Start': percstart, "%Owned": percrost, "OwnerChange":ownchange, "PRK": prk, "EstPoints":projallseason , "Avg": avg, "Last": last, "FFTeam": ffteam, "Week": week, "TimeUpdated": timeupdated},ignore_index=True)
        elif ac[1] in player[-8:] and player[-1] =="K":
            #Replace Team Acronym from Name String
            playpos = player.replace(ac[1], "")
            #Get to Injury status if aparent 
            pos = playpos[-2]
            pos = "K"
            if pos == "Q":
                #print("Questionable")
                inj = "Q"
                team = ac[1]
                obdf = obdf.append({'Slot': slot, 'Name': play, 'NFL_Team': team, 'Pos': pos,'Injury_Status': inj, 'Opponent':opp, "Kickoff": kickoff, 'Proj': proj, 'Score': score, 'OPRK':oprk, '%Start': percstart, "%Owned": percrost, "OwnerChange":ownchange, "PRK": prk, "EstPoints":projallseason , "Avg": avg, "Last": last, "FFTeam": ffteam, "Week": week, "TimeUpdated": timeupdated},ignore_index=True)
            else:
                #print(f'{playpos[:-1]} and {ac[1]}')
                play = playpos[:-1]
                inj = ""
                team = ac[1]
                obdf = obdf.append({'Slot': slot, 'Name': play, 'NFL_Team': team, 'Pos': pos,'Injury_Status': inj, 'Opponent':opp, "Kickoff": kickoff, 'Proj': proj, 'Score': score, 'OPRK':oprk, '%Start': percstart, "%Owned": percrost, "OwnerChange":ownchange, "PRK": prk, "EstPoints":projallseason , "Avg": avg, "Last": last, "FFTeam": ffteam, "Week": week, "TimeUpdated": timeupdated},ignore_index=True)
        elif ac[1] in player[-8:] and "D/ST" in player:
            #Replace Team Acronym from Name String
            playpos = player.replace(ac[1], "")
            play = playpos.replace(" D/ST", " ")
            pos = "D/ST"
            #print(play)
            team = ac[1]
            obdf = obdf.append({'Slot': slot, 'Name': play, 'NFL_Team': team, 'Pos': pos,'Injury_Status': inj, 'Opponent':opp, "Kickoff": kickoff, 'Proj': proj, 'Score': score, 'OPRK':oprk, '%Start': percstart, "%Owned": percrost, "OwnerChange":ownchange, "PRK": prk, "EstPoints":projallseason , "Avg": avg, "Last": last, "FFTeam": ffteam, "Week": week, "TimeUpdated": timeupdated},ignore_index=True)
        elif player == "Empty":
            pass
            play = "Empty"
            ac[1] = ""
            pos = None
    #bbdf = bbdf.append({'Slot': slot, 'Name': play, 'NFL_Team': team, 'Pos': pos,'Injury_Status': inj, 'Opponent':opp, "Kickoff": kickoff, 'Proj': proj, 'Score': score, 'OPRK':oprk, '%Start': percstart, "%Owned": percrost, "OwnerChange":ownchange, "PRK": prk, "EstPoints":projallseason , "Avg": avg, "Last": last, "FFTeam": ffteam, "Week": week, "TimeUpdated": timeupdated},ignore_index=True)
    #print(f'{slot}, {play}, {ac[1]}')
    
obdf

,Slot,Name,NFL_Team,Pos,Injury_Status,Opponent,Kickoff,Proj,Score,OPRK,%Start,%Owned,OwnerChange,PRK,EstPoints,Avg,Last,FFTeam,Week,TimeUpdated
0,QB,Patrick Mahomes,KC,QB,,Car,W 33-31,23.0,30.88,10th,68.9,100.0,0,1,237.98,26.4,30.88,Prostate Prophet,9,2020-11-10 10:04:56.171547
1,RB,Alvin Kamara,NO,RB,,@TB,W 38-3,14.7,10.9,9th,99.3,100.0,0,2,151.6,19.0,10.9,Prostate Prophet,9,2020-11-10 10:04:56.171547
2,RB,D'Andre Swift,Det,RB,,@Min,L 20-34,10.6,9.7,16th,69.3,95.8,+7.4,23,75.7,9.5,9.7,Prostate Prophet,9,2020-11-10 10:04:56.171547
3,WR,Tyreek Hill,KC,WR,,Car,W 33-31,11.5,24.1,6th,59.1,99.9,0,1,131.2,14.6,24.1,Prostate Prophet,9,2020-11-10 10:04:56.171547
4,WR,Robby Anderson,Car,WR,,@KC,L 31-33,8.9,6.3,4th,77.7,98.8,+3.8,20,81.4,9.0,6.3,Prostate Prophet,9,2020-11-10 10:04:56.171547
5,TE,Noah Fant,Den,TE,Q,@Atl,L 27-34,7.1,4.5,32nd,72.4,94.2,+13.5,12,48.9,7.0,4.5,Prostate Prophet,9,2020-11-10 10:04:56.171547
6,FLEX,Jerick McKinnon,SF,RB,,GB,L 17-34,9.5,12.8,31st,47.8,86.9,+14.6,19,79.4,8.8,12.8,Prostate Prophet,9,2020-11-10 10:04:56.171547
7,D/ST,Cardinals D/ST,Ari,D/ST,,Mia,L 31-34,7.2,2.0,13th,54.9,68.4,+53.2,14,39.0,4.9,2.0,Prostate Prophet,9,2020-11-10 10:04:56.171547
8,K,Ryan Succop,TB,K,,NO,L 3-38,8.7,4.0,12th,61.1,65.2,+44.5,9,79.0,8.8,4.0,Prostate Prophet,9,2020-11-10 10:04:56.171547
9,Bench,Kenny Golladay,Det,WR,Q,@Min,L 20-34,0.0,--,31st,23.3,97.1,-0.6,59,45.8,9.2,--,Prostate Prophet,9,2020-11-10 10:04:56.171547


In [44]:
obdf.to_csv("C:\\Users\\mpden\\Documents\\PythonScripts\\FFL\\Data\\most_recent\\ob.csv")
print("Oberle Collected")

Oberle Collected


# Get Dom

In [45]:
print("Collecting Dom")
time.sleep(8)
search_box = driver.find_element_by_xpath("//li[@class = 'teams NavSecondary__Item']")
search_box.click()


##Selecting Fantasy League
time.sleep(2)
leaguego = driver.find_element_by_partial_link_text('Vanilla')
leaguego.click()

# time.sleep(2)
# leaguego = driver.find_element_by_xpath("//span[@class='jsx-4290203343 week week-range']")
# leaguego.click()
print('Week 9')

# time.sleep(3)

Week 9


In [46]:
time.sleep(8)
page = driver.page_source
soup = BeautifulSoup(page, 'html.parser')
tables = soup.find_all("table")

table = tables[0]
tab_data = [[cell.text for cell in row.find_all(["th","td"])]
                        for row in table.find_all("tr")]
df = pd.DataFrame(tab_data)
gdf = df.drop([0,11,19])

gdf.columns = gdf.iloc[0]
gdf = gdf[1:]
#gdf.drop(columns=['action'])

gdf['Team'] = 'Vanilla Thunder'
gdf['Week'] = 9
gdf['Time'] = datetime.now()
gdf

1,SLOT,Player,opp,STATUS,proj,SCORE,OPRK,%ST,%ROST,+/-,PRK,FPTS,avg,LAST,Team,Week,Time
2,QB,Ben RoethlisbergerOPitQB,@Dal,W 24-19,17.9,25.04,22nd,47.9,84.1,+15.5,16,144.76,18.1,25.04,Vanilla Thunder,9,2020-11-10 10:05:43.049887
3,RB,David MontgomeryQChiRB,@Ten,L 17-24,12.5,2.2,24th,75.7,98.8,+5.1,18,80.4,8.9,2.2,Vanilla Thunder,9,2020-11-10 10:05:43.049887
4,RB,Justin JacksonQLACRB,LV,L 26-31,11.8,0.0,29th,59.1,92.6,+25.7,59,31.6,5.3,0.0,Vanilla Thunder,9,2020-11-10 10:05:43.049887
5,WR,Allen Robinson IIChiWR,@Ten,L 17-24,10.2,8.1,27th,95.8,99.8,+1.2,10,89.1,9.9,8.1,Vanilla Thunder,9,2020-11-10 10:05:43.049887
6,WR,JuJu Smith-SchusterPitWR,@Dal,W 24-19,8.1,15.3,30th,79.2,98.2,+2,26,69.9,8.7,15.3,Vanilla Thunder,9,2020-11-10 10:05:43.049887
7,TE,Evan EngramNYGTE,@Wsh,W 23-20,6.6,10.8,25th,69.1,89.6,+9.3,14,47.3,5.3,10.8,Vanilla Thunder,9,2020-11-10 10:05:43.049887
8,FLEX,Michael ThomasNOWR,@TB,W 38-3,10.6,5.1,9th,83.2,99.9,+0.1,150,6.8,3.4,5.1,Vanilla Thunder,9,2020-11-10 10:05:43.049887
9,D/ST,Buccaneers D/STTBD/ST,NO,L 3-38,4.3,-1.0,3rd,68.4,91.0,-2.8,4,75.0,8.3,-1.0,Vanilla Thunder,9,2020-11-10 10:05:43.049887
10,K,Younghoe KooAtlK,Den,W 34-27,9.2,12.0,28th,78.5,94.8,+17.4,1,93.0,11.6,12.0,Vanilla Thunder,9,2020-11-10 10:05:43.049887
12,Bench,Miles SandersQPhiRB,*BYE*,--,--,--,--,35.5,97.0,+0.7,26,68.5,13.7,--,Vanilla Thunder,9,2020-11-10 10:05:43.049887


In [47]:
nflteams = pd.read_csv('C:\\Users\\mpden\\Documents\\PythonScripts\\FFL\\Data\\static_tables\\teamacc.csv')
djdf = pd.DataFrame(columns = ['Slot', 'Name', 'NFL_Team', 'Pos','Injury_Status', 'Opponent', 'Kickoff', 'Proj', 'Score', 'OPRK', '%Start', "%Owned", "OwnerChange", "PRK", "EstPoints", "Avg", "Last", "FFTeam", "Week", "TimeUpdated"])

for row,col in gdf.iterrows():
    slot = col[0]
    player = col[1]
    opp = col[2]
    kickoff = col[3]
    proj = col[4]
    score = col[5]
    oprk = col[6]
    percstart = col[7]
    percrost = col[8]
    ownchange = col[9]
    prk = col[10]
    projallseason = col[11]
    avg = col[12]
    last = col[13]
    ffteam = col[14]
    week = col[15]
    timeupdated = col[16]
    for team, ac in nflteams.iterrows():
        if len(ac[1])>2 and ("D/ST" not in player) and player[-1] != 'K' and ac[1] in player[-8:]:
            #Replace Team Acronym from Name String
            playpos = player.replace(ac[1], "")
            #Get to Injury status if aparent 
            play = playpos[-3]
            pos = playpos[-2:]
            if play[-1] =='Q':
                #print(f'{playpos[:-3]} and {ac[1]} and Questionable')
                play = playpos[:-3]
                inj = "Q"
                team = ac[1]
                djdf = djdf.append({'Slot': slot, 'Name': play, 'NFL_Team': team, 'Pos': pos,'Injury_Status': inj, 'Opponent':opp, "Kickoff": kickoff, 'Proj': proj, 'Score': score, 'OPRK':oprk, '%Start': percstart, "%Owned": percrost, "OwnerChange":ownchange, "PRK": prk, "EstPoints":projallseason , "Avg": avg, "Last": last, "FFTeam": ffteam, "Week": week, "TimeUpdated": timeupdated},ignore_index=True)
            else:
                #print(f'{playpos[:-2]} and {ac[1]}')
                play = playpos[:-2]
                inj = ""
                team = ac[1]
                djdf = djdf.append({'Slot': slot, 'Name': play, 'NFL_Team': team, 'Pos': pos,'Injury_Status': inj, 'Opponent':opp, "Kickoff": kickoff, 'Proj': proj, 'Score': score, 'OPRK':oprk, '%Start': percstart, "%Owned": percrost, "OwnerChange":ownchange, "PRK": prk, "EstPoints":projallseason , "Avg": avg, "Last": last, "FFTeam": ffteam, "Week": week, "TimeUpdated": timeupdated},ignore_index=True)
        elif len(ac[1])<3 and ("D/ST" not in player) and player[-1] != 'K' and ac[1] in player[-8:]:
            #Replace Team Acronym from Name String
            playpos = player.replace(ac[1], "")
            #Get to Injury status if aparent 
            play = playpos[:-3]
            pos = playpos[-2:]
            if play[-1] =='Q':
                #print(f'{playpos[:-3]} and {ac[1]} and Questionable')
                play = playpos[:-3]
                inj = "Q"
                team = ac[1]
                djdf = djdf.append({'Slot': slot, 'Name': play, 'NFL_Team': team, 'Pos': pos,'Injury_Status': inj, 'Opponent':opp, "Kickoff": kickoff, 'Proj': proj, 'Score': score, 'OPRK':oprk, '%Start': percstart, "%Owned": percrost, "OwnerChange":ownchange, "PRK": prk, "EstPoints":projallseason , "Avg": avg, "Last": last, "FFTeam": ffteam, "Week": week, "TimeUpdated": timeupdated},ignore_index=True)
            else:
                #print(f'{playpos[:-2]} and {ac[1]}')
                play = playpos[:-2]
                inj = ""
                team = ac[1]
                djdf = djdf.append({'Slot': slot, 'Name': play, 'NFL_Team': team, 'Pos': pos,'Injury_Status': inj, 'Opponent':opp, "Kickoff": kickoff, 'Proj': proj, 'Score': score, 'OPRK':oprk, '%Start': percstart, "%Owned": percrost, "OwnerChange":ownchange, "PRK": prk, "EstPoints":projallseason , "Avg": avg, "Last": last, "FFTeam": ffteam, "Week": week, "TimeUpdated": timeupdated},ignore_index=True)
        elif ac[1] in player[-8:] and player[-1] =="K":
            #Replace Team Acronym from Name String
            playpos = player.replace(ac[1], "")
            #Get to Injury status if aparent 
            pos = playpos[-2]
            pos = "K"
            if pos == "Q":
                #print("Questionable")
                inj = "Q"
                team = ac[1]
                djdf = djdf.append({'Slot': slot, 'Name': play, 'NFL_Team': team, 'Pos': pos,'Injury_Status': inj, 'Opponent':opp, "Kickoff": kickoff, 'Proj': proj, 'Score': score, 'OPRK':oprk, '%Start': percstart, "%Owned": percrost, "OwnerChange":ownchange, "PRK": prk, "EstPoints":projallseason , "Avg": avg, "Last": last, "FFTeam": ffteam, "Week": week, "TimeUpdated": timeupdated},ignore_index=True)
            else:
                #print(f'{playpos[:-1]} and {ac[1]}')
                play = playpos[:-1]
                inj = ""
                team = ac[1]
                djdf = djdf.append({'Slot': slot, 'Name': play, 'NFL_Team': team, 'Pos': pos,'Injury_Status': inj, 'Opponent':opp, "Kickoff": kickoff, 'Proj': proj, 'Score': score, 'OPRK':oprk, '%Start': percstart, "%Owned": percrost, "OwnerChange":ownchange, "PRK": prk, "EstPoints":projallseason , "Avg": avg, "Last": last, "FFTeam": ffteam, "Week": week, "TimeUpdated": timeupdated},ignore_index=True)
        elif ac[1] in player[-8:] and "D/ST" in player:
            #Replace Team Acronym from Name String
            playpos = player.replace(ac[1], "")
            play = playpos.replace(" D/ST", " ")
            pos = "D/ST"
            #print(play)
            team = ac[1]
            djdf = djdf.append({'Slot': slot, 'Name': play, 'NFL_Team': team, 'Pos': pos,'Injury_Status': inj, 'Opponent':opp, "Kickoff": kickoff, 'Proj': proj, 'Score': score, 'OPRK':oprk, '%Start': percstart, "%Owned": percrost, "OwnerChange":ownchange, "PRK": prk, "EstPoints":projallseason , "Avg": avg, "Last": last, "FFTeam": ffteam, "Week": week, "TimeUpdated": timeupdated},ignore_index=True)
        elif player == "Empty":
            pass
            play = "Empty"
            ac[1] = ""
            pos = None
    #bbdf = bbdf.append({'Slot': slot, 'Name': play, 'NFL_Team': team, 'Pos': pos,'Injury_Status': inj, 'Opponent':opp, "Kickoff": kickoff, 'Proj': proj, 'Score': score, 'OPRK':oprk, '%Start': percstart, "%Owned": percrost, "OwnerChange":ownchange, "PRK": prk, "EstPoints":projallseason , "Avg": avg, "Last": last, "FFTeam": ffteam, "Week": week, "TimeUpdated": timeupdated},ignore_index=True)
    #print(f'{slot}, {play}, {ac[1]}')
    
djdf

,Slot,Name,NFL_Team,Pos,Injury_Status,Opponent,Kickoff,Proj,Score,OPRK,%Start,%Owned,OwnerChange,PRK,EstPoints,Avg,Last,FFTeam,Week,TimeUpdated
0,QB,Ben RoethlisbergerO,Pit,QB,,@Dal,W 24-19,17.9,25.04,22nd,47.9,84.1,+15.5,16,144.76,18.1,25.04,Vanilla Thunder,9,2020-11-10 10:05:43.049887
1,RB,David Montgomery,Chi,RB,Q,@Ten,L 17-24,12.5,2.2,24th,75.7,98.8,+5.1,18,80.4,8.9,2.2,Vanilla Thunder,9,2020-11-10 10:05:43.049887
2,RB,Justin Jackson,LAC,RB,Q,LV,L 26-31,11.8,0.0,29th,59.1,92.6,+25.7,59,31.6,5.3,0.0,Vanilla Thunder,9,2020-11-10 10:05:43.049887
3,WR,Allen Robinson II,Chi,WR,,@Ten,L 17-24,10.2,8.1,27th,95.8,99.8,+1.2,10,89.1,9.9,8.1,Vanilla Thunder,9,2020-11-10 10:05:43.049887
4,WR,JuJu Smith-Schuster,Pit,WR,,@Dal,W 24-19,8.1,15.3,30th,79.2,98.2,+2,26,69.9,8.7,15.3,Vanilla Thunder,9,2020-11-10 10:05:43.049887
5,TE,Evan Engram,NYG,TE,,@Wsh,W 23-20,6.6,10.8,25th,69.1,89.6,+9.3,14,47.3,5.3,10.8,Vanilla Thunder,9,2020-11-10 10:05:43.049887
6,FLEX,Michael Thomas,NO,WR,,@TB,W 38-3,10.6,5.1,9th,83.2,99.9,+0.1,150,6.8,3.4,5.1,Vanilla Thunder,9,2020-11-10 10:05:43.049887
7,D/ST,Buccaneers D/ST,TB,D/ST,,NO,L 3-38,4.3,-1.0,3rd,68.4,91.0,-2.8,4,75.0,8.3,-1.0,Vanilla Thunder,9,2020-11-10 10:05:43.049887
8,K,Younghoe Koo,Atl,K,,Den,W 34-27,9.2,12.0,28th,78.5,94.8,+17.4,1,93.0,11.6,12.0,Vanilla Thunder,9,2020-11-10 10:05:43.049887
9,Bench,Miles Sanders,Phi,RB,Q,*BYE*,--,--,--,--,35.5,97.0,+0.7,26,68.5,13.7,--,Vanilla Thunder,9,2020-11-10 10:05:43.049887


In [48]:
djdf.to_csv("C:\\Users\\mpden\\Documents\\PythonScripts\\FFL\\Data\\most_recent\\dj.csv")
print("Dom Collected")

Dom Collected


In [49]:
print('Finished')

Finished
